In [1]:
"""
Consolidates everything from all prior tutorials, except "euclidian stuff" (for that see 240410_kedar_euclidia...)
"""

'\nConsolidates everything from all prior tutorials, except "euclidian stuff" (for that see 240410_kedar_euclidia...)\n'

In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import matplotlib.pyplot as plt

# Load DFallPa dataset

In [3]:
##### Method: loading functrion"

In [4]:
from neuralmonkey.classes.population_mult import load_handsaved_wrapper, dfpa_match_chans_across_pa_each_bregion
from neuralmonkey.classes.population_mult import extract_single_pa


In [ ]:
# Method 1 - load a single DFallPA

# animal = "Pancho"
# date = 220614
# combine = True
# question = "CHAR_BASE_trial"
# version = "trial"

# animal = "Pancho"
# date = 220624
# combine = True
# question = "CHAR_BASE_stroke"
# version = "stroke"

animal = "Diego"
date = 231220
# date = 231122
combine = True
question = "CHAR_BASE_stroke"
version = "stroke"

DFallpa = load_handsaved_wrapper(animal, date, version=version, combine_areas=combine, question=question)


In [ ]:
from neuralmonkey.metadat.analy.anova_params import params_getter_euclidian_vars
from neuralmonkey.classes.population_mult import dfpa_concatbregion_preprocess_clean_bad_channels, dfpa_concatbregion_preprocess_wrapper

# LIST_VAR, LIST_VARS_OTHERS, LIST_CONTEXT, LIST_PRUNE_MIN_N_LEVS, LIST_FILTDICT = params_getter_euclidian_vars(question)

# Make a copy of all PA before normalization
dfpa_concatbregion_preprocess_wrapper(DFallpa, animal, date)

##### Do the standard preprocessing

In [ ]:
from neuralmonkey.scripts.analy_euclidian_chars_sp import behstrokes_preprocess_assign_col_bad_strokes
behstrokes_preprocess_assign_col_bad_strokes(DFallpa, animal, date)


In [ ]:
from neuralmonkey.scripts.analy_euclidian_chars_sp import preprocess_dfallpa_prune_chans
from neuralmonkey.scripts.analy_euclidian_chars_sp import preprocess_dfallpa_prune_chans_hand_coded
SAVEDIR_ANALYSIS = "/tmp"
preprocess_dfallpa_prune_chans(DFallpa, animal, date, SAVEDIR_ANALYSIS)
preprocess_dfallpa_prune_chans_hand_coded(DFallpa, animal, date)

# Other stuff

##### Make sure PIG uses clean strokes

In [ ]:
PA = DFallpa["pa"].values[0]
dflab = PA.Xlabels["trials"]

from pythonlib.tools.pandastools import grouping_print_n_samples
# grouping_print_n_samples(dflab, ["task_kind", "shape_semantic", "clust_sim_max_colname"])
grouping_print_n_samples(dflab, ["task_kind", "shape_semantic"])


In [ ]:
dflab["charclust_shape_seq"]

##### Check, why bug in trialtimes?

In [ ]:
PA = DFallpa["pa"].values[0]
dflab = PA.Xlabels["trials"]


In [ ]:
dflab["trialcode_scal"]

##### Diego, consolidate his shapes

In [ ]:
PA = DFallpa["pa"].values[0]
dflab = PA.Xlabels["trials"]

In [ ]:
sorted(dflab["shape_semantic"].unique())

In [ ]:
from pythonlib.drawmodel.tokens import MAP_SHAPESEM_TO_SHAPESEMGROUP, map_shsem_to_new_shsem

# Consolidate Diego shapes
dflab["shape_semantic"] = [map_shsem_to_new_shsem[sh] if sh in map_shsem_to_new_shsem.keys() else sh for sh in dflab["shape_semantic"]]

# Sanity check that this doesnt change shape_sem_grp
for _, row in dflab.iterrows():
    assert MAP_SHAPESEM_TO_SHAPESEMGROUP[row["shape_semantic"]] == row["shape_semantic_grp"]


### Fixing code that checks for drifty neurons

In [ ]:
path1 = "/lemur2/lucas/Dropbox/SCIENCE/FREIWALD_LAB/DATA/Xuan/DFallpa-Diego-231218-stroke-kilosort_if_exists-norm=None-combine=True-t1=-1.0-t2=1.8-quest=CHAR_BASE_stroke.pkl"
path2 = "/lemur2/lucas/Dropbox/SCIENCE/FREIWALD_LAB/DATA/Xuan/DFallpa-Diego-231218-trial-kilosort_if_exists-norm=None-combine=True-t1=-1.0-t2=1.8-quest=CHAR_BASE_trial.pkl"

import pickle
with open(path1, "rb") as f:
    dfallpa1 = pickle.load(f)

with open(path2, "rb") as f:
    dfallpa2 = pickle.load(f)    

In [ ]:
print(dfallpa1["pa"].values[1].Chans)
print(dfallpa2["pa"].values[1].Chans)
print(DFallpa["pa"].values[1].Chans)

In [ ]:
import pickle
# path = "/lemur2/lucas/analyses/recordings/main/EXPORTED_BEH_DATA/DS/char_strokes_clusters/Diego/basis=Diego-shapes=None/231122/DS_data.pkl"
# path = "/lemur2/lucas/analyses/recordings/main/EXPORTED_BEH_DATA/DS/char_strokes_clusters/Diego/basis=Diego-shapes=None/231128/DS_data.pkl"
path = "/lemur2/lucas/analyses/recordings/main/EXPORTED_BEH_DATA/DS/char_strokes_clusters/Diego/basis=Diego-shapes=None/231206/DS_data.pkl"
with open(path, "rb") as f:
    df = pickle.load(f)

In [ ]:
df[df["task_kind"]=="character"]["shape_semantic"].value_counts()

# Targeted dim reduction --> state space plots

### (1) Compute, extract, save, make plots

In [ ]:
DFallpa = DFallpa[:1]

In [ ]:
from neuralmonkey.scripts.analy_euclidian_chars_sp import targeted_dim_reduction_wrapper
import os

for var_effect in ["shape_semantic_grp", "shape_semantic"]:
    LIST_VARIABLES = [
        ["task_kind", "stroke_index_is_first", "loc_on_clust_cat", "reach_angle_binned", var_effect],
    ]
    # LIST_VARIABLES = [
    #     ["task_kind", "stroke_index_is_first", "loc_on_clust_cat", var_effect],
    #     ["task_kind", "stroke_index_is_first", var_effect],
    #     ["stroke_index_is_first", var_effect],
    # ]

    for variables in LIST_VARIABLES:
        variables_is_cat = [True for _ in range(len(variables))]

        SAVEDIR_ANALYSIS = f"/tmp/CHAR_SP_TARGETED_DR/vareff={var_effect}--varsregr={'|'.join(variables)}"
        os.makedirs(SAVEDIR_ANALYSIS, exist_ok=True)

        print(SAVEDIR_ANALYSIS)
        DFDIST = targeted_dim_reduction_wrapper(DFallpa, animal, date, SAVEDIR_ANALYSIS,
                                                variables, variables_is_cat, var_effect = var_effect)
        

### (2) Load mult dates, expts and plot

In [ ]:
from neuralmonkey.scripts.analy_shape_invariance_all_plots_SP import euclidian_time_resolved_fast_shuffled_mult_reload, _euclidian_time_resolved_fast_shuffled_mult_scatter_plots_params
import pandas as pd
from pythonlib.tools.pandastools import append_col_with_grp_index
import os
import pickle

# Dataset params
animal = "Pancho"
combine = True

# Plotting params
yvar = "dist_yue_diff"
MAKE_HEATMAP_DATAPTS = False # This can be slow

# hard coded params
analysis_kind = "char_sp_00_stroke"
var_other = "task_kind"
list_date = _euclidian_time_resolved_fast_shuffled_mult_scatter_plots_params(analysis_kind, animal, var_other)

for var_effect in ["shape_semantic_grp", "shape_semantic"]:
    # var_effect = "shape_semantic_grp"

    LIST_VARIABLES = [
        ["task_kind", "stroke_index_is_first", "loc_on_clust_cat", "reach_angle_binned", var_effect],
        ["task_kind", "stroke_index_is_first", "loc_on_clust_cat", var_effect],
        ["task_kind", "stroke_index_is_first", var_effect],
        ["stroke_index_is_first", var_effect],
    ]

    for variables in LIST_VARIABLES:
        variables_is_cat = [True for _ in range(len(variables))]

        # variables = ["task_kind", "stroke_index_is_first", "loc_on_clust_cat", "reach_angle_binned", var_effect]
        # variables_is_cat = [True for _ in range(len(variables))]

        SAVEDIR_MULT = f"/lemur2/lucas/analyses/recordings/main/euclidian_char_sp/targeted_pca/MULT-combine={combine}/var={var_effect}--varsregr={'|'.join(variables)}/{animal}"
        os.makedirs(SAVEDIR_MULT, exist_ok=True)

        ### LOAD, IF EXISTS
        path = f"{SAVEDIR_MULT}/DFDIST_FINAL.pkl"
        if os.path.exists(path):
            DFDIST = pd.read_pickle(path)
            with open(f"{SAVEDIR_MULT}/DICT_DFDISTS.pkl", "rb") as f:
                DICT_DFDISTS = pickle.load(f)
        else:
            ### (1) load each dates data
            list_dfdist = []
            for date in list_date:
                SAVEDIR_ANALYSIS = f"/lemur2/lucas/analyses/recordings/main/euclidian_char_sp/targeted_pca/{animal}-{date}-combine={combine}/var={var_effect}--varsregr={'|'.join(variables)}"

                print("Loading: ", SAVEDIR_ANALYSIS)

                DFDIST = pd.read_pickle(f"{SAVEDIR_ANALYSIS}/DFDIST.pkl")

                # DFDIST = append_col_with_grp_index(DFDIST, [f"{var_effect}_same", "task_kind_same"], "same-shape|tk")
                # DFDIST["tk_12_sorted"] = [tuple(sorted([x["task_kind_1"], x["task_kind_2"]])) for _, x in DFDIST.iterrows()]
                # DFDIST["sifirst_12_sorted"] = [tuple(sorted([x["stroke_index_is_first_1"], x["stroke_index_is_first_2"]])) for _, x in DFDIST.iterrows()]
                # # One plot for each pair of task kinds that are different.

                # # Create a new variable -- (task_kind, stroke is first)
                # for i in [1, 2]:
                #     DFDIST = append_col_with_grp_index(DFDIST, [f"task_kind_{i}", f"stroke_index_is_first_{i}"], f"tk_sifirst_{i}")
                # DFDIST["tk_sifirst_same"] = DFDIST["tk_sifirst_1"] == DFDIST["tk_sifirst_2"]
                # DFDIST["tk_sifirst_12"] = DFDIST["tk_sifirst_1"] + "|" + DFDIST["tk_sifirst_2"]
                # DFDIST["tk_sifirst_sorted"] = [tuple(sorted([x["tk_sifirst_1"], x["tk_sifirst_2"]])) for _, x in DFDIST.iterrows()]

                # Collect
                DFDIST["animal"] = animal
                DFDIST["date"] = date

                list_dfdist.append(DFDIST)
            DFDIST = pd.concat(list_dfdist).reset_index(drop=True)

            ### (2) Condition the dataframe 
            # Create a new variable -- (task_kind, stroke is first)
            for i in [1, 2]:
                print(i)
                DFDIST = append_col_with_grp_index(DFDIST, [f"task_kind_{i}", f"stroke_index_is_first_{i}"], f"tk_sifirst_{i}")
            DFDIST["tk_sifirst_same"] = DFDIST["tk_sifirst_1"] == DFDIST["tk_sifirst_2"]
            DFDIST["tk_sifirst_12"] = DFDIST["tk_sifirst_1"] + "|" + DFDIST["tk_sifirst_2"]
            DFDIST["tk_sifirst_sorted"] = [tuple(sorted([x["tk_sifirst_1"], x["tk_sifirst_2"]])) for _, x in DFDIST.iterrows()]
            DFDIST = append_col_with_grp_index(DFDIST, [f"{var_effect}_same", "task_kind_same"], "same-shape|tk")

            ### (3) Split into specific paired datasets
            from neuralmonkey.scripts.analy_euclidian_chars_sp import targeted_dim_reduction_post_split_into_specicic_datasets
            assert len(DFDIST["animal"].unique())==1
            DICT_DFDISTS = {}
            for date in list_date:
                dfdist = DFDIST[DFDIST["date"] == date].reset_index(drop=True)

                dict_dfdists = targeted_dim_reduction_post_split_into_specicic_datasets(dfdist, var_effect)

                for key, df in dict_dfdists.items():
                    print(key, " --> n=", len(df))
                    if key not in DICT_DFDISTS:
                        DICT_DFDISTS[key] = [df]
                    else:
                        DICT_DFDISTS[key].append(df)
            # convert each to a df
            DICT_DFDISTS = {k:pd.concat(list_df).reset_index(drop=True) for k, list_df in DICT_DFDISTS.items()}

            ### (4) Save the final split dfdists...
            DFDIST.to_pickle(f"{SAVEDIR_MULT}/DFDIST_FINAL.pkl")
            with open(f"{SAVEDIR_MULT}/DICT_DFDISTS.pkl", "wb") as f:
                pickle.dump(DICT_DFDISTS, f)

        ### (5) Plots
        from pythonlib.tools.pandastools import grouping_plot_n_samples_conjunction_heatmap
        import seaborn as sns
        from pythonlib.tools.pandastools import aggregGeneral
        from pythonlib.tools.pandastools import plot_45scatter_means_flexible_grouping
        from pythonlib.tools.pandastools import plot_subplots_heatmap, grouping_append_and_return_inner_items_good
        from pythonlib.tools.plottools import savefig

        # Plot an exmaple
        # for tksipair in DICT_DFDISTS.keys(): # key = ('character|0', 'prims_single|1')
        for tksipair in [('character|0', 'prims_single|1'), ('character|1', 'prims_single|1')]: # key = ('character|0', 'prims_single|1')
            # for twind_scal in DFDIST["twind_scal"].unique():
            for twind_scal in [(-0.5, -0.05)]: # twind_scal = (-0.5, -0.05)
                for npcs_keep in DFDIST["npcs_keep"].unique(): # npcs_keep = 2
                # for npcs_keep in [2, 4, 6]: # npcs_keep = 2
                    savedir = f"{SAVEDIR_MULT}/tksipair={tksipair}-twindscal={twind_scal}-npcs={npcs_keep}"
                    os.makedirs(savedir, exist_ok=True)
                    
                    # Extract this dfdist
                    dfdist = DICT_DFDISTS[tksipair]
                    dfdist = dfdist[(dfdist["twind_scal"] == twind_scal) & (dfdist["npcs_keep"] == npcs_keep)].reset_index(drop=True)
                    dfdist = append_col_with_grp_index(dfdist, ["bregion", "same-shape|tk"], "br-ss|tk")
                    dfdist_agg = aggregGeneral(dfdist, ["bregion", "date", "same-shape|tk"], ["dist_norm", "dist_yue_diff"])

                    # grouping_plot_n_samples_conjunction_heatmap(dfdist, "shape_semantic_grp_1", "shape_semantic_grp_2" )
                    fig = grouping_plot_n_samples_conjunction_heatmap(dfdist, f"{var_effect}_1", f"{var_effect}_2", ["date"]);
                    savefig(fig, f"{savedir}/counts.pdf")
                    plt.close("all")

                    fig = sns.catplot(data=dfdist, x="bregion", y=yvar, hue="same-shape|tk", col="date", col_wrap=6,
                                kind="bar", aspect=1)
                    savefig(fig, f"{savedir}/catplot-1.pdf")

                    fig = sns.catplot(data=dfdist, x="bregion", y=yvar, hue="same-shape|tk", col="date", col_wrap=6,
                                alpha=0.5, jitter=True, aspect=1)
                    savefig(fig, f"{savedir}/catplot-2.pdf")

                    # Summary plot
                    fig = sns.catplot(data=dfdist, x="bregion", y=yvar, hue="same-shape|tk", kind="bar", aspect=1)
                    savefig(fig, f"{savedir}/catplot-summary-3.pdf")

                    fig = sns.catplot(data=dfdist_agg, x="bregion", y=yvar, hue="same-shape|tk", kind="bar", aspect=1)
                    savefig(fig, f"{savedir}/catplot-agg-1.pdf")

                    fig = sns.catplot(data=dfdist_agg, x="bregion", y=yvar, hue="same-shape|tk", alpha=0.5, jitter=True, aspect=1)
                    savefig(fig, f"{savedir}/catplot-agg-2.pdf")

                    # Scatter
                    _, fig = plot_45scatter_means_flexible_grouping(dfdist, "same-shape|tk", "1|0", "0|1", None, var_value=yvar, var_datapt="bregion")
                    savefig(fig, f"{savedir}/scatter-1.pdf")

                    # Scatter
                    _, fig = plot_45scatter_means_flexible_grouping(dfdist, "same-shape|tk", "1|0", "0|1", "date", var_value=yvar, var_datapt="bregion")
                    savefig(fig, f"{savedir}/scatter-2.pdf")

                    plt.close("all")

                    # Plot each pair of conditions -- low-level plot
                    if MAKE_HEATMAP_DATAPTS:
                        grpdict = grouping_append_and_return_inner_items_good(dfdist, ["date"])
                        for grp, inds in grpdict.items():
                            dfdist_this = dfdist.iloc[inds].reset_index(drop=True)
                            dfdist_this = dfdist_this[dfdist_this["same-shape|tk"] != "1|1"].reset_index(drop=True) # to remove 1/4 of subplots.

                            fig, axes = plot_subplots_heatmap(dfdist_this, f"{var_effect}_1", f"{var_effect}_2", yvar, "br-ss|tk", False, True, ncols=6)    

                            savefig(fig, f"{savedir}/heatmap_all_conditions-{grp}.pdf")
                            plt.close("all")


### Further devo

In [ ]:
idx = 1
PA = DFallpa["pa"].values[idx]
dflab = PA.Xlabels["trials"]

dflab["gap_from_prev_angle_b"]

dflab["gap_from_prev_angle_binned"].value_counts()
assert sum(dflab["gap_from_prev_angle_binned"].isna())==0
dflab["reach_angle_binned"] = [f"angle_{x}" for x in dflab["gap_from_prev_angle_binned"]]

In [ ]:
##### TODO: how much of total variance is accounted for by this subspace?

# Original method [fast shuff] but regressing out first stroke effect

In [ ]:
if False:
    # Only keep shapes that exist across all conditions
    from pythonlib.tools.pandastools import append_col_with_grp_index, extract_with_levels_of_conjunction_vars_helper

    VAR_SHAPE = "shape_semantic"

    savedir = "/tmp"
    PA = DFallpa["pa"].values[1]
    dflab = PA.Xlabels["trials"]
    dflab = append_col_with_grp_index(dflab, ["task_kind", "stroke_index_is_first"], "tk_sifirst")
    levels_var = list(dflab["tk_sifirst"].unique())
    path_counts_pre = f"{savedir}/counts_pre.pdf"
    path_counts = f"{savedir}/counts.pdf"
    dfout, dict_dfthis = extract_with_levels_of_conjunction_vars_helper(dflab, "tk_sifirst", [VAR_SHAPE], 4, path_counts, 
                                                lenient_allow_data_if_has_n_levels=None, levels_var=levels_var, plot_counts_also_before_prune_path=path_counts_pre)
    PA = PA.slice_by_dim_indices_wrapper("trials", dfout["_index"].tolist())

In [ ]:
# First, plot state space [HEATMAP code]
from neuralmonkey.scripts.analy_euclidian_chars_sp import plot_heatmap_firing_rates_all_wrapper
import os

DEBUG_bregion = "PMv"
# DEBUG_subspace_projection = "task_shape_si"
# DEBUG_subspace_projection = "task_shape"
var_conj = "tk_sifirst"
list_twind_scal = [(-0.6, 0.4)]
LIST_SUBSPACE_PROJECTION = ["task_shape"]
# LIST_SUBSPACE_PROJECTION = ["task_shape_si"]

heatmap_quick_version = True
do_heatmap = False
also_plot_clean_version = True

for regress_out_first_stroke in [False, True]:
    # for consolidate_diego_shapes_actually in [False, True]:
    for consolidate_diego_shapes_actually in [True]:
                
        # regress_out_first_stroke = True
        # LIST_PRUNE_VERSION = [None, "sp_char", "sp_char_1plus", "sp_pig", "sp_pig_1plus", "sp_pigchar_1plus"]
        LIST_PRUNE_VERSION = [None, "sp_char", "sp_pig"]
        keep_only_shapes_that_exist_across_all_contexts = True
        # consolidate_diego_shapes_actually = True
        SAVEDIR_ANALYSIS = f"/tmp/CHAR_SP_HETAMAP-regress={regress_out_first_stroke}-consolidateshapes={consolidate_diego_shapes_actually}"
        os.makedirs(SAVEDIR_ANALYSIS, exist_ok=True)

        plot_heatmap_firing_rates_all_wrapper(DFallpa, SAVEDIR_ANALYSIS, animal, date, 
                                                DEBUG_skip_drawings=True, DEBUG_bregion=DEBUG_bregion,
                                                DEBUG_subspace_projection=None,
                                                var_conj=var_conj,
                                                LIST_PRUNE_VERSION=LIST_PRUNE_VERSION,
                                                list_twind_scal=list_twind_scal,
                                                LIST_SUBSPACE_PROJECTION=LIST_SUBSPACE_PROJECTION,
                                                heatmap_quick_version=heatmap_quick_version,
                                                DO_HACK=regress_out_first_stroke,
                                                do_heatmap=do_heatmap, 
                                                keep_only_shapes_that_exist_across_all_contexts=keep_only_shapes_that_exist_across_all_contexts,
                                                consolidate_diego_shapes_actually=consolidate_diego_shapes_actually,
                                                also_plot_clean_version=also_plot_clean_version)


In [ ]:
# Reorder shapes to match

In [ ]:
# Develop code to remove effect of frist stroke.
PA = DFallpa["pa"].values[0]
PA.Xlabels["trials"]


In [ ]:
# Second, score euclidian distance
from neuralmonkey.scripts.analy_euclidian_chars_sp import euclidian_time_resolved_fast_shuffled
import os
SAVEDIR_ANALYSIS = f"/tmp/CHAR_SP_EUCL_FAST"
os.makedirs(SAVEDIR_ANALYSIS, exist_ok=True)

# DFallpa = DFallpa.loc[[1],:]
DEBUG_bregion_list = ["PMv", "preSMA"]
HACK = True
DO_REGRESS_HACK = True
euclidian_time_resolved_fast_shuffled(DFallpa, animal, date, SAVEDIR_ANALYSIS, DO_RSA_HEATMAPS=True, 
                                      HACK = HACK, DEBUG_bregion_list=DEBUG_bregion_list, DO_REGRESS_HACK=DO_REGRESS_HACK)


##### For Diego exmaple in revision, reorder shapes to match the previous plots in first submissiong

In [ ]:
# This is because now using shape_semantic, not shape_semantic_grp, and so the shape names are slightly different

In [ ]:
pa = DFallpa["pa"].values[0]
dflab = pa.Xlabels["trials"]
sorted(list(dflab["shape_semantic"].unique()))

shapes_in_order = ['arcdeep-DD-DD',
 'arcdeep-RR-RR',
 'arcdeep-UU-UU',
 'V-DD-DD',
 'V-RR-RR',
 'V-UU-UU',
 'Lcentered-DL-DL',
 'Lcentered-DR-DR',
 'Lcentered-UL-UL',
 'Lcentered-UR-UR',
 'usquare-DD-DD',
 'usquare-RR-RR',
 'usquare-UU-UU',
 'zigzagSq-LL-0.0',
 'zigzagSq-LL-1.0',
 'zigzagSq-UU-0.0',
 'circle-XX-XX',
 'line-LL-LL',
 'line-UL-UL',
 'line-UR-UR',
 'line-UU-UU',
 'squiggle3-LL-0.0',
 'squiggle3-LL-1.0',
 'squiggle3-UU-0.0']

map_shape_to_shapeordered = {}
for i, sh in enumerate(shapes_in_order):
    if i<10:
        map_shape_to_shapeordered[sh] = f"0{i}-{sh}"
    else:
        map_shape_to_shapeordered[sh] = f"{i}-{sh}"

map_shape_to_shapeordered
# dflab["shape_semantic"] = [map_shape_to_shapeordered[sh] for sh in dflab["shape_semantic"]]

In [ ]:
DFallpa

In [ ]:
##### Quick use computing of r2 scalar value
PA = DFallpa["pa"].values[5]

In [ ]:
# (1) Collect r2 over all neurosn
var = "shape_semantic"
list_r2 = []
for idx_chan in range(len(PA.Chans)):
    r2 = PA.metrics_scalar_compute_r2(idx_chan, var)
    list_r2.append(r2)

print("Range of r2: ", np.percentile(list_r2, [2.5, 50, 97.5]))

In [ ]:
# (2) For a given neuron, split by var_other, and compute r2 for each of those levels.
idx_chan = 0
list_var = ["shape_semantic"]
ms = PA.dataextract_metrics_scalar(idx_chan, list_var)
r2 = PA.metrics_scalar_compute_r2(idx_chan, var)

var = "shape_semantic"
var_other = "gridloc"

list_pa, list_levels = PA.split_by_label("trials", [var_other])
res = []
for levo, pa in zip(list_levels, list_pa):
    r2 = pa.metrics_scalar_compute_r2(idx_chan, var)
    res.append({
        "level_other":levo,
        "r2":r2
    })
dfres = pd.DataFrame(res)    
dfres

##### Devo -- cleaning up strokes even more

##### Remove strokes based on motor params

In [ ]:
from neuralmonkey.scripts.analy_euclidian_chars_sp import behstrokes_map_clustshape_to_thresh, behstrokes_extract_char_clust_sim

assert DFallpa["event"].unique().tolist() == ["00_stroke"], "with trials, there is ambiguity bceuae of mult strokes per trials..."

### First, use one PA to extract beh and label each trial as good bad based on beh strokes.
PA = DFallpa["pa"].values[0]
ds = behstrokes_extract_char_clust_sim(PA, animal, date, savedir=None, PLOT=False)



In [ ]:
##### Do the pruning in preprocess_pa.

In [ ]:
dflab = PA.Xlabels["trials"]

##### Remove char strokes that are too different from sp strokes

In [ ]:
# Load all the pairwise stroke distances (previously extracted)

In [ ]:
PA = DFallpa["pa"].values[0]
dflab = PA.Xlabels["trials"]

In [ ]:
import pickle

var_shape = "shape_semantic_grp"
SAVEDIR = f"/lemur2/lucas/analyses/recordings/main/euclidian_char_sp/motor_comparison/{animal}-{date}-combine={combine}-var={var_shape}"
nmax_for_pairs = None


savedir = f"{SAVEDIR}/{var_shape}/paired_distance-nmax={nmax_for_pairs}"

# for shape in dflab[var_shape].unique():
shape = "Lcentered-UR-UR"
path = f"{savedir}/alldists-{shape}.pkl"
with open(path, "rb") as f:
    obj = pickle.load(f)



In [ ]:
Cl = obj["Cl"]

In [ ]:
Cl.plot_heatmap_data()

In [ ]:
# For each char stroke, get its average distance to each sp stroke
import numpy as np
dist_each_char = np.mean(Cl.Xinput, axis=1)

fig, ax = plt.subplots( figsize=(5, 12))
ax.set_xlim([0, 1])
ax.plot(dist_each_char, obj["trialcodes_char"], "ok")


In [ ]:
#TODO: decide on a good threshold. Not that obvious, since some char may match some SP. use vareage over sp? 


In [ ]:
# from neuralmonkey.scripts.analy_euclidian_chars_sp import preprocess_pa
# preprocess_pa(animal, date, PA, "/tmp", None, shape_var = var_shape, 
#                   n_min_trials_per_shape=N_MIN_TRIALS_PER_SHAPE,
#                   plot_counts_heatmap_savepath=None, plot_drawings=True, remove_chans_fr_drift=False,
#                   subspace_projection=None, twind_analy=None, tbin_dur=None, tbin_slide=None, NPCS_KEEP=None, 
#                   raw_subtract_mean_each_timepoint=False, remove_singleprims_unstable=False,
#                   remove_trials_with_bad_strokes=True, subspace_projection_fitting_twind=None,
#                   skip_dim_reduction=False, scalar_or_traj="traj",
#                   remove_trials_too_fast=None)

### Regressing out other variables, not just "first stroke"


In [ ]:
tbin_dur = 0.1
tbin_slide = 0.05
twind = twind_analy = (-1, 0.6)
PA = PA.regress_neuron_task_variables_subtract_from_activity(tbin_dur, tbin_slide, twind)


In [ ]:
dflab = PA.Xlabels["trials"]
dflab["reach_angle_class"] = [f"reach_{x}" for x in dflab["gap_from_prev_angle_binned"]]

In [ ]:
i = 1
pathis = PA.slice_by_dim_indices_wrapper("times", [i])

var_effect = "shape_semantic_grp"
variables = ["stroke_index_is_first", "task_kind", "reach_angle_class", var_effect]
variables_is_cat = [True for _ in range(len(variables))]
dfcoeff, dfbases, _, _ = pathis.regress_neuron_task_variables_all_chans(variables, 
                                                                variables_is_cat, PLOT_COEFF_HEATMAP=True)

# This is the value you should subtract from all cases with first stroke = True
a = dfcoeff["C(stroke_index_is_first)[T.True]"].values[:, None] # (nchans, 1)

dflab = PA.Xlabels["trials"]
b = dflab["stroke_index_is_first"].astype(int).values[:, None] # (ntrials, 1)



In [ ]:
dfbases

In [ ]:
[x.shape for x in dfbases["Xpca"]]

In [ ]:
Xpca = dfbases[dfbases["var_subspace"] == "reach_angle_class"]["Xpca"].values[0]
explained_variance_ratio_ = dfbases[dfbases["var_subspace"] == "reach_angle_class"]["explained_variance_ratio_"].values[0]

In [ ]:
Xpca.shape

Xp


In [ ]:
fig, ax = plt.subplots()
ax.plot(explained_variance_ratio_)

In [ ]:
# Use PCA to get main axees for a given categorical variable.


In [ ]:
dfcoeff

# [GOOD] Careful controlling of motor behavior

In [ ]:
from neuralmonkey.scripts.analy_euclidian_chars_sp import behstrokes_preprocess_plot_motor_stats_similarity
import os

var_shape = "shape_semantic_grp"
SAVEDIR = "/tmp/CHAR_SP_MOTOR"
os.makedirs(SAVEDIR, exist_ok=True)
nmax_for_pairs = None
df_paireddist = behstrokes_preprocess_plot_motor_stats_similarity(DFallpa, animal, date, var_shape, SAVEDIR, 
                                                                  nmax_for_pairs=nmax_for_pairs)

In [ ]:
PA = DFallpa["pa"].values[0]
dflab = PA.Xlabels["trials"]

dflab["clust_sim_max_GOOD"].value_counts()

In [ ]:
dflab["shape_semantic_grp"]

In [ ]:
var_shape = "shape_semantic_grp"
dflab[var_shape]

In [ ]:
from neuralmonkey.scripts.analy_euclidian_chars_sp import behstrokes_preprocess_plot_motor_stats_similarity


In [ ]:
from pythonlib.tools.pandastools import grouping_print_n_samples
grouping_print_n_samples(dflab, ["shape_semantic_grp", "shape"])

In [ ]:
grouping_print_n_samples(ds.Dat, ["shape_semantic_grp", "clust_sim_max_colname"])

In [ ]:
df = ds.Dat[ds.Dat["task_kind"] == "character"].reset_index(drop=True)
grouping_print_n_samples(df, ["shape_semantic", "shape_semantic"]);

In [ ]:
# First, make plots of all pairwise distances between trials, and their drawings.
# Second, make sure the distance metric is fine
# -- Check how I modified the trajecotry distance to be better (see line 3378 in psychometric_singleprims)
# Third, relate the behavioral distance to the neural distance.
# Fourth, prune data to keep just close distance?

In [ ]:
### [Alterantive, but decided teh above was fine] Load the original dataset

from pythonlib.dataset.dataset import load_dataset_notdaily_helper, load_dataset_daily_helper

D = load_dataset_daily_helper(animal, date)

In [ ]:
D.Dat["charclust_shape_seq_scores"]

In [ ]:
from pythonlib.dataset.dataset_strokes import preprocess_dataset_to_datstrokes
DS = preprocess_dataset_to_datstrokes(D, "chars_clusters_without_reloading")


In [ ]:
DS.Dat["gap_from_prev_dur"]

# Controlling for motor, see diff in neural

### M1 vs. PMv, showing that PMv cares more about the category

In [ ]:
import os
SAVEDIR = f"/tmp/MOTOR"
# twind_analy = (-0.5, 0.)
list_twind_analy = [(-0.5, 0.), (-0.2, 0.5)]
list_frac = [(0, 0.5), (0, 1.)]
list_bregion = DFallpa["bregion"].unique().tolist()

In [ ]:
dflab = PA.Xlabels["trials"]
dflab["index_datapt"]

In [ ]:
from neuralmonkey.scripts.analy_euclidian_chars_sp import motor_encoding_score_wrapper
motor_encoding_score_wrapper(DFallpa, animal, date, SAVEDIR)

In [ ]:
# keep only cases with enough data

dfres



In [ ]:
##### Older, regression methods

In [ ]:
from pythonlib.tools.pandastools import convert_var_to_categorical_mult_columns


In [ ]:
# Find dist values 
dfdist_neural

In [ ]:
# [Devo code] Overlay regression lines.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# Perform Linear Regression
# X = df['X'].values.reshape(-1, 1)
# Y = df['Y'].values.reshape(-1, 1)

X = data[xvar].values.reshape(-1, 1)
Y = data[yvar].values.reshape(-1, 1)

# restrict beh distance to remove outliers
xmax = np.percentile(X, [98])
# ymax = np.percentile(Y, [98])
# Y = Y[Y<ymax]

fig, ax = plt.subplots()
ax.hist(X, bins=100)
ax.axvline(xmax, color="r")

inds_keep = X<xmax
X = X[inds_keep, None]
Y = Y[inds_keep, None]

model = LinearRegression()
model.fit(X, Y)

# Get Predictions
Y_pred = model.predict(X)

# Plot the Data and Best-Fit Line
plt.figure(figsize=(8, 6))
plt.scatter(X, Y, color='blue', label='Data Points')
plt.plot(X, Y_pred, color='red', linewidth=2, label='Best-Fit Line')

# Customize the Plot
plt.title('Linear Regression Best-Fit Line')
plt.xlabel('X')
plt.ylabel('Y')
plt.legend()
plt.show()


In [ ]:
# Do regression to compare same vs. diff

from pythonlib.tools.pandastools import grou


##### Better method -- parametrize the initial movement in the stroke

In [ ]:
from neuralmonkey.scripts.analy_euclidian_chars_sp import preprocess_pa
import os

In [ ]:
# Also get:
# - old state space, with location, angle
# - substrokes -- not the extractioN (which fails for chars) but the features code.
# - Motor kinematics (regression) - /home/lucas/code/neuralmonkey/neuralmonkey/notebooks/230208_motor_kinematics.ipynb
# - 

In [ ]:
import os
SAVEDIR = "/tmp/MOTOR_PARAMETRIZES"
os.makedirs(SAVEDIR, exist_ok=True)


In [ ]:
# Bin the angles
from pythonlib.tools.vectools import bin_angle_by_direction
bin_angle_by_direction(angles, num_angle_bins=8)


In [ ]:
fig, ax = plt.subplots()
ax.plot(angles, np.sin(angles), "xk")
ax.plot(angles, np.cos(angles), "xr")
ax.plot(angles, np.sin(angles)**2 + np.cos(angles)**2, "xb")


In [ ]:
var_shape = "shape_semantic"

def preprocess_and_get_motor(PA, prune_version, subspace_projection, savedir):
    """
    Do dim reduction etc for PA.
    Also extract all relevant motor features.

    RETURNS:
    - copy of PA.
    
    """
    ### 
    n_min_trials_per_shape = 4
    plot_drawings = False
    # tbin_dur = 0.2
    # tbin_slide = 0.02
    tbin_dur = 0.1
    tbin_slide = 0.05

    # Do umap on timecourse
    NPCS_KEEP = 8
    raw_subtract_mean_each_timepoint = False

    subspace_projection_fitting_twind = (-0.8, 0.3)
    # twind_analy = (-0.2, 0.1)
    twind_analy = (-0.35, -0.05)
    # To get scalar
    scalar_or_traj = "scal"
    PA = preprocess_pa(animal, date, PA, savedir, prune_version, 
                        n_min_trials_per_shape=n_min_trials_per_shape, shape_var=var_shape, plot_drawings=plot_drawings,
                        remove_chans_fr_drift=False,
                        subspace_projection=subspace_projection, 
                        twind_analy=twind_analy, tbin_dur=tbin_dur, tbin_slide=tbin_slide, NPCS_KEEP=NPCS_KEEP,
                        raw_subtract_mean_each_timepoint=raw_subtract_mean_each_timepoint,
                        subspace_projection_fitting_twind=subspace_projection_fitting_twind,
                        remove_singleprims_unstable=True,
                        scalar_or_traj=scalar_or_traj, 
                        consolidate_diego_shapes=False)

    if True:
        PA.behavior_strokes_kinematics_stats()
    else:
        # Replacing below
        # Dim rediction of beh to good scalar plots
        PA.behavior_extract_strokes_to_dflab()
        dflab = PA.Xlabels["trials"]
        strokes = dflab["strok_beh"].tolist()

        ### MOTOR PARAMETERS
        # get the initial velocity (angle and magnitude)
        from pythonlib.dataset.dataset_strokes import DatStrokes
        DS = DatStrokes()
        
        ### For each stroke, compute some features
        import numpy as np
        from pythonlib.tools.stroketools import sliceStrokes, slice_strok_by_frac_bounds
        from pythonlib.tools.stroketools import feature_velocity_vector_angle_norm

        # (1) For initial angle, take start of each stroke 
        twind = [0, 0.15] # sec
        strokes_sliced = sliceStrokes(strokes, twind, time_is_relative_each_onset=True, assert_no_lost_strokes=True)

        # - velocity vector
        angles = [] 
        norms = []
        for strok in strokes_sliced:
            _, a, n = feature_velocity_vector_angle_norm(strok)
            angles.append(a)
            norms.append(n)
        
        # - circularity
        from pythonlib.drawmodel.features import strokeCircularity
        fraclow = 0
        frachigh = 0.5
        strokes_sliced = [slice_strok_by_frac_bounds(s, fraclow, frachigh) for s in strokes]
        # strokes_sliced = slice_strok_by_frac_bounds(strokes, twind, time_is_relative_each_onset=True, assert_no_lost_strokes=True)

        # circularities = strokeCircularity(strokes)
        circularities = strokeCircularity(strokes_sliced)

        # - location of onset.
        # (reach angle, relative to on location)
        onsets_x = [strok[0, 0] for strok in strokes]
        onsets_y = [strok[0, 1] for strok in strokes]

        # Bin the angles
        from pythonlib.tools.vectools import bin_angle_by_direction
        angles_binned = bin_angle_by_direction(angles, num_angle_bins=8)

        # Put motor variables back into dflab
        dflab["motor_angle"] = angles
        # dflab["motor_angle_sin"] = angles

        dflab["motor_angle_binned"] = angles_binned
        
        dflab["gap_from_prev_angle_binned"] = bin_angle_by_direction(dflab["gap_from_prev_angle"].values, num_angle_bins=8)

        dflab["motor_norm"] = norms

        dflab["motor_circ"] = circularities

        dflab["motor_onsetx"] = onsets_x

        dflab["motor_onsety"] = onsets_y

        PA.Xlabels["trials"] = dflab

    return PA

In [ ]:
import numpy as np
# Extract PA, preprocessed
# bregion = "PMv"

# prune_version = "sp_char_0"
# subspace_projection = "task_shape_si"

# prune_version = "sp_char_0"
# subspace_projection = "shape_all"
# var_shape = "shape_semantic_grp"

for prune_version, subspace_projection in [
    # ("sp_char", "shape_all"),
    # ("sp_pig_char", "shape_all"),
    ("sp_char_0", "task_shape_si"),
]:
    # prune_version = "sp_char"
    # subspace_projection = "shape_all"

    # prune_version = "sp_pig_char"
    # subspace_projection = "shape_all"

    for bregion in DFallpa["bregion"].unique().tolist():

        savedir = f"{SAVEDIR}/prune={prune_version}-subspace={subspace_projection}/{bregion}"
        os.makedirs(savedir, exist_ok=True)

        ### First, get beh distance between strokes. This takes a min or two so do it out here.
        PA = extract_single_pa(DFallpa, bregion=bregion, which_level="stroke", event="00_stroke")

        PA = preprocess_and_get_motor(PA, prune_version, subspace_projection, savedir)

        if False:
            dflab = PA.Xlabels["trials"]

            # To compute any feature
            # - strokeclass.extract

            # To compute velocity
            # - dataset_strokes.feature_com
            # To bin and plot
            # - substrokes.features_motor...
            shapes = dflab[var_shape].tolist()
            import random
            nplot = 10
            inds = sorted(random.sample(list(range(len(strokes))), nplot))

            titles = [f"{i}|{shapes[i]}|a={angles[i]:.2f}|n={norms[i]:.0f}|c={circularities[i]:.2f}|x={onsets_x[i]:.0f}|y={onsets_y[i]:.0f}" for i in inds]
            
            # Plot exmaples

            DS.plot_multiple_strok([strokes[i] for i in inds], titles=titles, overlay=False, size_per_sublot=4.8)
            DS.plot_multiple_strok([strokes_sliced[i] for i in inds], titles=titles, overlay=False, size_per_sublot=4.8)
            # Plot state space, coloring by all the variables, and overlaying stroke examples

            # PA.plot_state_space_good_wrapper(savedir, LIST_VAR, LIST_VARS_OTHERS=None, LIST_FILTDICT=None, LIST_PRUNE_MIN_N_LEVS=None,
            #                                       time_bin_size = 0.05, PLOT_CLEAN_VERSION = False, 
            #                                       nmin_trials_per_lev=None, list_dim_timecourse=None, list_dims=None)


        X = PA.X
        X = np.mean(X, axis=2).T
        dflab = PA.Xlabels["trials"]
        from neuralmonkey.analyses.state_space_good import trajgood_plot_colorby_splotby_scalar_WRAPPER

        # LIST_VAR = [
        #     var_shape, 
        #     "motor_angle_binned",
        #     "motor_angle", 
        #     "motor_norm",
        #     "motor_circ",
        #     "motor_onsetx",
        #     "motor_onsety", 
        #     "gap_from_prev_angle",
        #     var_shape, 
        #     "motor_angle_binned",
        #     "motor_angle", 
        #     "motor_norm",
        #     "motor_circ",
        #     "motor_onsetx",
        #     "motor_onsety", 
        #     "gap_from_prev_angle",
        #     var_shape, 
        #     "motor_angle_binned",
        #     "motor_angle", 
        #     "motor_norm",
        #     "motor_circ",
        #     "motor_onsetx",
        #     "motor_onsety", 
        #     "gap_from_prev_angle",
        #     var_shape, 
        #     "motor_angle_binned",
        #     "motor_angle", 
        #     "motor_norm",
        #     "motor_circ",
        #     "motor_onsetx",
        #     "motor_onsety",
        #     "gap_from_prev_angle"]
        # LIST_VARS_OTHERS = [
        #     None,
        #     None,
        #     None,
        #     None,
        #     None,
        #     None,
        #     None,
        #     None,
        #     ("task_kind",),
        #     ("task_kind",),
        #     ("task_kind",),
        #     ("task_kind",),
        #     ("task_kind",),
        #     ("task_kind",),
        #     ("task_kind",),
        #     ("task_kind",),    
        #     (var_shape,),
        #     (var_shape,),
        #     (var_shape,),
        #     (var_shape,),
        #     (var_shape,),
        #     (var_shape,),
        #     (var_shape,),
        #     (var_shape,),
        #     "motor_angle_binned",
        #     "motor_angle_binned",
        #     "motor_angle_binned",
        #     "motor_angle_binned",
        #     "motor_angle_binned",
        #     "motor_angle_binned",
        #     "motor_angle_binned",
        #     "motor_angle_binned",
            
        # ]


        LIST_VAR = [
            var_shape, 
            var_shape, 
            var_shape, 
            var_shape, 
            var_shape, 
            var_shape, 
            "motor_angle_binned",
            "motor_angle", 
            "motor_norm",
            "motor_circ",
            "motor_onsetx",
            "motor_onsety", 
            "gap_from_prev_angle",
            ]
        
        LIST_VARS_OTHERS = [
            None,
            ("task_kind",),
            ("motor_angle_binned", "task_kind"),
            ("motor_angle_binned"),
            ("gap_from_prev_angle_binned", "task_kind"),
            ("gap_from_prev_angle_binned"),
            (var_shape, "task_kind"),
            (var_shape, "task_kind"),
            (var_shape, "task_kind"),
            (var_shape, "task_kind"),
            (var_shape, "task_kind"),
            (var_shape, "task_kind"),
            (var_shape, "task_kind"),
        ]

        # list_dims = [(0,1), (1,2), (2,3), (3,4)]
        list_dims = [(0,1), (1,2)]
        for var, vars_others in zip(LIST_VAR, LIST_VARS_OTHERS):
            trajgood_plot_colorby_splotby_scalar_WRAPPER(X, dflab, var, savedir, vars_others, list_dims)

        plt.close("all")

##### Quick analysis of euclidian distance, using binned beh features

In [ ]:
SAVEDIR = "/tmp/MOTOR_EUCLIDIAN"
os.makedirs(SAVEDIR, exist_ok=True)

In [ ]:
from pythonlib.tools.plottools import savefig
import os

In [ ]:
prune_version = "sp_pig_char"
subspace_projection = "shape_all"
shape_var = "shape_semantic_grp"
subspace_projection_fitting_twind = (-0.4, 0.4)
twind_analy = (-0.2, 0.1)
var_other = "gap_from_prev_angle_binned"

# prune_version = "sp_char_0"
# subspace_projection = "task_shape_si"
# shape_var = "shape_semantic"
# subspace_projection_fitting_twind = (-0.8, 0.3)
# twind_analy = (-0.35, -0.05)
# var_other = "motor_angle_binned"

# prune_version = "sp_char_0"
# subspace_projection = "task_shape_si"
# shape_var = "shape_semantic"
# subspace_projection_fitting_twind = (-0.2, 0.5)
# twind_analy = (-0.1, 0.25)
# var_other = "motor_angle_binned"

for bregion in ["PMv", "M1", "PMd", "SMA"]:
    savedir = f"{SAVEDIR}/{bregion}"
    os.makedirs(savedir, exist_ok=True)

    ### 
    n_min_trials_per_shape = 4
    plot_drawings = False
    # tbin_dur = 0.2
    # tbin_slide = 0.02
    tbin_dur = 0.1
    tbin_slide = 0.05

    # Do umap on timecourse
    NPCS_KEEP = 8
    raw_subtract_mean_each_timepoint = False


    ### First, get beh distance between strokes. This takes a min or two so do it out here.
    PA = extract_single_pa(DFallpa, bregion=bregion, which_level="stroke", event="00_stroke")

    # To get scalar
    scalar_or_traj = "traj"
    PA = preprocess_pa(animal, date, PA, savedir, prune_version, 
                        n_min_trials_per_shape=n_min_trials_per_shape, shape_var=shape_var, plot_drawings=plot_drawings,
                        remove_chans_fr_drift=False,
                        subspace_projection=subspace_projection, 
                            twind_analy=twind_analy, tbin_dur=tbin_dur, tbin_slide=tbin_slide, NPCS_KEEP=NPCS_KEEP,
                            raw_subtract_mean_each_timepoint=raw_subtract_mean_each_timepoint,
                            subspace_projection_fitting_twind=subspace_projection_fitting_twind,
                            remove_singleprims_unstable=True,
                            scalar_or_traj=scalar_or_traj)

    # Dim rediction of beh to good scalar plots
    PA.behavior_strokes_kinematics_stats()
    if False:
        PA.behavior_extract_strokes_to_dflab()
        dflab = PA.Xlabels["trials"]
        strokes = dflab["strok_beh"].tolist()

        ### MOTOR PARAMETERS
        # get the initial velocity (angle and magnitude)
        from pythonlib.dataset.dataset_strokes import DatStrokes
        DS = DatStrokes()

        ### For each stroke, compute some features
        import numpy as np
        from pythonlib.tools.stroketools import sliceStrokes
        from pythonlib.tools.stroketools import feature_velocity_vector_angle_norm

        # (1) For initial angle, take start of each stroke 
        twind = [0, 0.15] # sec
        strokes_sliced = sliceStrokes(strokes, twind, time_is_relative_each_onset=True, assert_no_lost_strokes=True)

        # - velocity vector
        angles = [] 
        norms = []
        for strok in strokes_sliced:
            _, a, n = feature_velocity_vector_angle_norm(strok)
            angles.append(a)
            norms.append(n)

        # - circularity
        from pythonlib.drawmodel.features import strokeCircularity

        # twind = [0, 0.15] # sec
        # strokes_sliced = sliceStrokes(strokes, twind, time_is_relative_each_onset=True, assert_no_lost_strokes=True)
        circularities = strokeCircularity(strokes)

        # - location of onset.
        # (reach angle, relative to on location)
        onsets_x = [strok[0, 0] for strok in strokes]
        onsets_y = [strok[0, 1] for strok in strokes]

        # Bin the angles
        from pythonlib.tools.vectools import bin_angle_by_direction
        angles_binned = bin_angle_by_direction(angles, num_angle_bins=6)

        # Put motor variables back into dflab
        dflab["motor_angle"] = angles
        dflab["motor_angle_binned"] = angles_binned
        dflab["motor_norm"] = norms
        dflab["motor_circ"] = circularities
        dflab["motor_onsetx"] = onsets_x
        dflab["motor_onsety"] = onsets_y

        # dflab["gap_from_prev_angle_binned"] = bin_angle_by_direction(dflab["gap_from_prev_angle"].values, num_angle_bins=8)
        # dflab["gap_from_prev_angle_binned"] = bin_angle_by_direction(dflab["gap_from_prev_angle"].values, num_angle_bins=8)
        dflab["gap_from_prev_angle_binned"] = bin_angle_by_direction(dflab["gap_from_prev_angle"].values, num_angle_bins=4)


    # #### COMPUTE EUCLIDIAN
    # var_context_same = "task_kind"
    # savedir = "/tmp/RSA"
    # os.makedirs(savedir, exist_ok=True)
    # from neuralmonkey.analyses.euclidian_distance import timevarying_compute_fast_to_scalar
    # vars_group = [shape_var, "motor_angle_binned"]
    # # vars_group = [shape_var, "gap_from_prev_angle_binned"]
    # # DFDIST = PA.dataextractwrap_distance_between_groups(vars_group, "traj_to_scal")
    # context_dict = {"same":["task_kind"], "diff":[]}
    # dfdist, Cldist = timevarying_compute_fast_to_scalar(PA, vars_group, savedir, var_context_same=var_context_same)
    # import seaborn as sns
    # sns.catplot(data=dfdist, x="same-shape_semantic_grp|motor_angle_binned", y="dist_yue_diff", alpha=0.5, jitter=True)
    # sns.catplot(data=dfdist, x="same-shape_semantic_grp|motor_angle_binned", y="dist_yue_diff", kind="bar")
    # import seaborn as sns
    # sns.catplot(data=dfdist, x="same-shape_semantic_grp|motor_angle_binned", y="dist_yue_diff", alpha=0.5, jitter=True)
    # sns.catplot(data=dfdist, x="same-shape_semantic_grp|motor_angle_binned", y="dist_yue_diff", kind="bar")


    # from pythonlib.tools.vectools import bin_angle_by_direction
    # angles_binned = bin_angle_by_direction(angles, num_angle_bins=8)

    # # Put motor variables back into dflab
    # dflab["motor_angle"] = angles
    # dflab["motor_angle_binned"] = angles_binned
    # dflab["motor_norm"] = norms
    # dflab["motor_circ"] = circularities
    # dflab["motor_onsetx"] = onsets_x
    # dflab["motor_onsety"] = onsets_y


    # dflab["this"] = bin_angle_by_direction(dflab["gap_from_prev_angle"].values, num_angle_bins=8)


    #### COMPUTE EUCLIDIAN
    # var_context_same = None
    os.makedirs(savedir, exist_ok=True)
    from neuralmonkey.analyses.euclidian_distance import timevarying_compute_fast_to_scalar
    vars_group = [shape_var, var_other]
    # vars_group = ["shape_semantic_grp", "this"]
    # DFDIST = PA.dataextractwrap_distance_between_groups(vars_group, "traj_to_scal")
    dfdist, Cldist = timevarying_compute_fast_to_scalar(PA, vars_group, savedir)

    import seaborn as sns
    fig = sns.catplot(data=dfdist, x=f"same-{shape_var}|{var_other}", y="dist_yue_diff", alpha=0.5, jitter=True)
    savefig(fig, f"{savedir}/catplot-1.pdf")
    fig = sns.catplot(data=dfdist, x=f"same-{shape_var}|{var_other}", y="dist_yue_diff", kind="bar")
    savefig(fig, f"{savedir}/catplot-2.pdf")

    plt.close("all")


In [ ]:
### A single PC space for motor
from neuralmonkey.analyses.state_space_good import dimredgood_pca

features = []


In [ ]:
PA.X.shape

##### Regression -- condition on each shape, and predict a variable using the neural activity


In [ ]:
SAVEDIR = "/tmp/MOTOR_REGR"
# os.makedirs()

In [ ]:
# Extract data
prune_version = "sp_pig_char"
subspace_projection = "shape_all"
bregion = "FP"

savedir = f"{SAVEDIR}/prune={prune_version}-subspace={subspace_projection}/{bregion}"
os.makedirs(savedir, exist_ok=True)

### First, get beh distance between strokes. This takes a min or two so do it out here.
PA = extract_single_pa(DFallpa, bregion=bregion, which_level="stroke", event="00_stroke")

PA = preprocess_and_get_motor(PA, prune_version, subspace_projection, savedir)



In [ ]:
PA.X.shape

In [ ]:
from pythonlib.tools.pandastools import grouping_append_and_return_inner_items_good
from pythonlib.tools.vectools import linearize_angles_around_mean

var_beh = "motor_angle"
vars_grp = [var_shape]

ndims = NPCS_KEEP

dflab = PA.Xlabels["trials"]
X = PA.X
assert X.shape[2]==1
grpdict = grouping_append_and_return_inner_items_good(dflab, vars_grp)

for grp, inds in grpdict.items():
    
    x = X[:ndims, inds, 0].T # (dims, trials)
    y_beh = dflab.loc[inds, var_beh].values

    # normalize the beh
    if var_beh in ["motor_angle"]:
        _, y_beh, _, _ = linearize_angles_around_mean(y_beh)
    
    # regression
    

In [ ]:
import statsmodels.formula.api as smf


In [ ]:
dflab.loc[inds, var_beh].values

In [ ]:
x.shape, y_beh.shape

In [ ]:
X.shape, y.shape

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression
import numpy as np

# Generate synthetic data for regression
# X, y = make_regression(n_samples=500, n_features=5, noise=10, random_state=42)

# Initialize the Linear Regression model
model = LinearRegression()

# Cross-validate the model
cv_results = cross_validate(
    model, x, y_beh, cv=2,  # 5-fold cross-validation
    scoring=['r2', 'neg_mean_squared_error'],  # Specify scoring metrics
    return_train_score=True  # Include training scores
)

# Display Results
print("Cross-Validation Results:")
for metric in cv_results:
    print(f"{metric}: {cv_results[metric]}")

# Summarize Mean and Std for Test Scores
print("\nMean R2 (Test):", np.mean(cv_results['test_r2']))
print("Mean MSE (Test):", -np.mean(cv_results['test_neg_mean_squared_error']))  # MSE is negated for interpretation


In [ ]:
# SUMMARY:
# - good, clearly see that M1 high beh, lower shape, and PMv opposite, if do quick euclidian test, and also
# if inspect the scatter plots.
# --- PMd, looks like PMv. I think this would go away once include reach direction too.

# TODO:
# Plots:
# - color angle circular.
# - binnned reach angle (this explains PMd?)
# - PCA on beh variables --> then bin and color in plots. 

# Euclidian:
# - already did above, but was hacky. 
# - If do this, then should optimize how many bins. And maybe first do PCA of beh, and then bin.

# Regression:
# - This is best? for asking about relationship with beh, conditioned on shape.
# - Should actually make the input data be the angles. Analogous to how the input data for euclidian distance is shape categories.

# Pairwise analysis
# - This might also be good. This is the previous method (motor_encoding_score_wrapper), but replacing the motor score there with pairwise distance (e.g., angle)
# - TODO: make those plots and see if it looks good.

# SUMMARY:
# Actually, euclidian is very related to pairwise analysis
# - decision for me is whether to use binned beh, or to do pairwise between raw beh, and then split into close and far.

# Regression is better if you think there is systematic, simple relationship (e.g., linear or circular)    

# THOUGHTS:
# - probably first try pairwise distances ()

# ALSO:
# - some other notes on ways to analyse the pairwise distance (in macbook).


In [ ]:
### OLD
# Do slice
from pythonlib.tools.stroketools import slice_strok_by_frac_bounds
strokes = [slice_strok_by_frac_bounds(strok, frac_bounds_stroke[0], frac_bounds_stroke[1]) for strok in strokes]

# Get pairwise distnace between all strokes.
from pythonlib.dataset.dataset_strokes import DatStrokes
ds = DatStrokes()
labels = [tuple(x) for x in dflab.loc[:, vars_group].values.tolist()]

list_distance_ver = ["euclid_vels_2d"]
CldistStroke = ds.distgood_compute_beh_beh_strok_distances(strokes, strokes, list_distance_ver, labels_rows_dat=labels,
                                                labels_cols_feats=labels, label_var=vars_group, clustclass_rsa_mode=True)
CldistStroke._Xinput = 1-CldistStroke._Xinput # So that ranges from [0, 1] where 0 is best.
dfdist_stroke = CldistStroke.rsa_dataextract_with_labels_as_flattened_df(keep_only_one_direction_for_each_pair=False, plot_heat=False, exclude_diagonal=True)


In [ ]:
# First, for each stroke, extract the onset (substroke)



In [ ]:
# Plot state space, coloring by different kinematic metrics.


In [ ]:
# Find cases of (same shape, diff motor) and (diff shape, similar motor).


In [ ]:
##### 

### Compare beh strokes

In [ ]:
PA.behavior_extract_strokes_to_dflab(trial_take_first_stroke=True)


In [ ]:
dflab = PA.Xlabels["trials"]

In [ ]:
from pythonlib.tools.pandastools import grouping_append_and_return_inner_items_good
shape_var = "shape_semantic_grp"
grpdict = grouping_append_and_return_inner_items_good(dflab, [shape_var, "task_kind"])

list_shape = dflab[shape_var].unique().tolist()

In [ ]:
# Given two set of strokes, score how similar they are.
tk1 = "prims_single"
tk2 = "character"
for shape in list_shape:
    key1 = (shape, tk1)
    key2 = (shape, tk2)
    if key1 not in grpdict.keys():
        print("Key not found: ", key1)
        continue
    if key2 not in grpdict.keys():
        print("Key not found: ", key2)
        continue
    
    strokes1 = dflab.iloc[grpdict[key1]]["strok_beh"].tolist()
    strokes2 = dflab.iloc[grpdict[key2]]["strok_beh"].tolist()



In [ ]:

from pythonlib.dataset.dataset_strokes import DatStrokes
ds = DatStrokes()
Cl = ds.distgood_compute_beh_beh_strok_distances(strokes1, strokes2, label_var=shape_var)



In [ ]:
from neuralmonkey.scripts.analy_euclidian_chars_sp import behstrokes_preprocess_assign_col_bad_strokes, preprocess_pa
behstrokes_preprocess_assign_col_bad_strokes(DFallpa, animal, date)

In [ ]:
from neuralmonkey.scripts.analy_euclidian_chars_sp import behstrokes_preprocess_assign_col_bad_strokes, preprocess_pa

pa = DFallpa["pa"].values[5]
preprocess_pa(animal, date, pa, "/tmp", None, remove_trials_too_fast=True)

In [ ]:
behstrokes_extract_char_clust_sim(PA, animal, date, savedir=None, PLOT=False)

In [ ]:
PA = DFallpa["pa"].values[0]
behstrokes_extract_char_clust_sim(PA, animal, date, "/tmp", True)

# [0] Do state space and euclidian

##### First, prune PA to just good data

In [ ]:
from pythonlib.tools.pandastools import append_col_with_grp_index
import seaborn as sns
from pythonlib.tools.plottools import savefig

SAVEDIR_ANALYSIS = "/tmp/CHAR_SP_FINAL"
import os

In [ ]:
# Prune to just the DFallpa for testing.


In [ ]:
######### PARAMS
n_min_trials_per_shape = 5
LIST_NPCS_KEEP = [4,6,2]
PLOT_EACH_REGION = True

### State space plots
LIST_VAR = [
    "shape_semantic",
    "shape_semantic",
    "shape_semantic",
]
LIST_VARS_OTHERS = [
    ["task_kind", "stroke_index"],
    ["task_kind", "stroke_index"],
    ["task_kind", "stroke_index"],
]
LIST_CONTEXT = [
    {"same":["stroke_index"], "diff":["task_kind"]},
    {"same":["stroke_index"], "diff":["task_kind"]},
    {"same":["stroke_index"], "diff":["task_kind"]},
]
LIST_PRUNE_MIN_N_LEVS = [2 for _ in range(len(LIST_VAR))]
LIST_FILTDICT = [
    {"task_kind":["prims_single", "character"], "stroke_index":[0]},
    {"task_kind":["prims_single", "character"]},
    {"task_kind":["prims_single", "prims_on_grid"]},
    ]

for twind_analy in [(0.05, 0.25), (-0.05, 0.35), (0.1, 0.2)]:
    for subspace_projection in ["shape_prims_single", "pca"]:
        for prune_version in ["sp_char_0", "sp_char"]:
            for NPCS_KEEP in LIST_NPCS_KEEP:
                for raw_subtract_mean_each_timepoint in [False, True]:
                    SAVEDIR = f"{SAVEDIR_ANALYSIS}/subspc={subspace_projection}-prunedat={prune_version}-npcs={NPCS_KEEP}-subtr={raw_subtract_mean_each_timepoint}-twind={twind_analy}"
                    os.makedirs(SAVEDIR, exist_ok=True)

                    assert False
                    PLOT_STATE_SPACE = NPCS_KEEP == max(LIST_NPCS_KEEP)
                    run(animal, date, DFallpa, SAVEDIR, subspace_projection, prune_version, NPCS_KEEP, 
                            raw_subtract_mean_each_timepoint, n_min_trials_per_shape,
                            PLOT_EACH_REGION, PLOT_STATE_SPACE,
                            LIST_VAR, LIST_VARS_OTHERS, LIST_FILTDICT, LIST_PRUNE_MIN_N_LEVS, twind_analy)


##### Run, a single time

In [ ]:
from neuralmonkey.scripts.analy_euclidian_chars_sp import run
import os


In [ ]:
# prune_version = "sp_char_0"
# # subspace_projection = "shape_prims_single"
# subspace_projection = None
# twind_analy = (-0.15, 0.2)
# tbin_dur = 0.1
# tbin_slide = 0.02

prune_version = "sp_char_0"
subspace_projection = "shape_prims_single"
# subspace_projection = None
twind_analy = (0.05, 0.25)
tbin_dur = 0.1
tbin_slide = 0.05

NPCS_KEEP = None
raw_subtract_mean_each_timepoint = False
n_min_trials_per_shape = 5
PLOT_EACH_REGION = False
PLOT_STATE_SPACE = False
LIST_VAR = ["shape_semantic"]
LIST_VARS_OTHERS = [
    ["task_kind", "stroke_index"],
]
LIST_CONTEXT = [
    {"same":["stroke_index"], "diff":["task_kind"]},
]
LIST_PRUNE_MIN_N_LEVS = [2 for _ in range(len(LIST_VAR))]
LIST_FILTDICT = [
    {"task_kind":["prims_single", "character"], "stroke_index":[0]},
    ]

SAVEDIR = f"/tmp/TEST-{twind_analy}"
os.makedirs(SAVEDIR, exist_ok=True)

run(animal, date, DFallpa, SAVEDIR, subspace_projection, prune_version, NPCS_KEEP, 
        raw_subtract_mean_each_timepoint, n_min_trials_per_shape,
        PLOT_EACH_REGION, PLOT_STATE_SPACE,
        LIST_VAR, LIST_VARS_OTHERS, LIST_FILTDICT, LIST_PRUNE_MIN_N_LEVS, twind_analy,
        tbin_dur = tbin_dur, tbin_slide = tbin_slide)


In [ ]:
# Get time-series of eucl distance




##### Run, for a single bregion

In [ ]:
from neuralmonkey.classes.population_mult import extract_single_pa
from neuralmonkey.scripts.analy_euclidian_chars_sp import preprocess_pa
import os

In [ ]:
SAVEDIR = f"/tmp/TEST/{animal}-{date}"
os.makedirs(SAVEDIR, exist_ok=True)

In [ ]:
bregion = "preSMA_a"    
prune_version = "sp_char_0"
subspace_projection = "shape_prims_single"
# subspace_projection = None
raw_subtract_mean_each_timepoint = False
remove_drift = False

In [ ]:
twind_analy = (-0.15, 0.2)
tbin_dur = 0.1
tbin_slide = 0.02


In [ ]:
n_min_trials_per_shape = 5

# Run
PA = extract_single_pa(DFallpa, bregion, which_level="stroke", event="00_stroke")

savedir = f"{SAVEDIR}/preprocess"
os.makedirs(savedir, exist_ok=True)
plot_drawings = False
PA = preprocess_pa(animal, date, PA, savedir, prune_version, 
                    n_min_trials_per_shape=n_min_trials_per_shape, plot_drawings=plot_drawings,
                    remove_chans_fr_drift=remove_drift,
                    subspace_projection=subspace_projection, 
                           twind_analy=twind_analy, tbin_dur=tbin_dur, tbin_slide=tbin_slide, NPCS_KEEP=NPCS_KEEP)



In [ ]:
# Optionally, keep specific chans
# chans_keep = [1053, 1054]
chans_keep = [1044, 1049,  1053, 1054, 1057, 1059, 1062]
PA = PA.slice_by_dim_values_wrapper("chans", chans_keep)


In [ ]:
if True:
    pa = PA.slice_by_dim_values_wrapper("times", twind_analy)
    pa = pa.agg_by_time_windows_binned(tbin_dur, tbin_slide)

else:
    subspace_projection = "shape_prims_single"

    if subspace_projection == "pca":
        dim_red_method = "pca"
        superv_dpca_params={
            "superv_dpca_var":None,
            "superv_dpca_vars_group":None,
            "superv_dpca_filtdict":None,
        }
    elif subspace_projection == "shape_prims_single":
        dim_red_method = "superv_dpca"
        superv_dpca_params={
            "superv_dpca_var":"shape_semantic",
            "superv_dpca_vars_group":None,
            "superv_dpca_filtdict":{"task_kind":["prims_single"]}
        }
    elif subspace_projection == "shape_PIG_stroke0":
        # PIG (0)  
        dim_red_method = "superv_dpca"
        superv_dpca_params={
            "superv_dpca_var":"shape_semantic",
            "superv_dpca_vars_group":None,
            "superv_dpca_filtdict":{"task_kind":["prims_on_grid"], "stroke_index":[0]}
        }
    elif subspace_projection == "shape_char_stroke0":
        # Char  
        dim_red_method = "superv_dpca"
        superv_dpca_params={
            "superv_dpca_var":"shape_semantic",
            "superv_dpca_vars_group":None,
            "superv_dpca_filtdict":{"task_kind":["character"], "stroke_index":[0]}
        }
    else:
        print(subspace_projection)
        assert False
        

    ### New, cleaner method, taking all pairwise distances between trials
    savedir = f"{SAVEDIR}/each_region/{bregion}"
    os.makedirs(savedir, exist_ok=True)

    # (1) First, dim reduction
    superv_dpca_var = superv_dpca_params['superv_dpca_var']
    superv_dpca_vars_group = superv_dpca_params['superv_dpca_vars_group']
    superv_dpca_filtdict = superv_dpca_params['superv_dpca_filtdict']


    dim_red_method = "pca"
    twind_analy = (-0.15, 0.3)
    tbin_dur = 0.1
    tbin_slice = 0.01
    NPCS_KEEP = 10
    _, PAredu = PA.dataextract_dimred_wrapper("traj", dim_red_method, savedir, 
                                    twind_analy, tbin_dur=tbin_dur, tbin_slide=tbin_slice, 
                                    NPCS_KEEP = NPCS_KEEP,
                                    dpca_var = None, dpca_vars_group = None, dpca_filtdict=None, 
                                    dpca_proj_twind = twind_analy, 
                                    raw_subtract_mean_each_timepoint=raw_subtract_mean_each_timepoint,
                                    umap_n_components=None, umap_n_neighbors=None)
    

    pa = PAredu


# [2] Quick analyses of euclidian distances

In [ ]:
vars_group = ["task_kind", "shape_semantic"]
version = "traj"
DFDIST = pa.dataextractwrap_distance_between_groups(vars_group, version)
DFDIST

In [ ]:
from pythonlib.tools.pandastools import append_col_with_grp_index
# DFDIST = append_col_with_grp_index(DFDIST, ["shape_semantic_1", "shape_semantic_2"], "shape_semantic_same")
# DFDIST = append_col_with_grp_index(DFDIST, ["task_kind_1", "task_kind_2"], "task_kind_same")

DFDIST["task_kind_same"] = DFDIST["task_kind_1"] == DFDIST["task_kind_2"]
DFDIST["shape_semantic_same"] = DFDIST["shape_semantic_1"] == DFDIST["shape_semantic_2"]

DFDIST = append_col_with_grp_index(DFDIST, ["task_kind_1", "task_kind_2"], "task_kind_12")


DFDIST = append_col_with_grp_index(DFDIST, ["task_kind_same", "shape_semantic_same"], "same-task|shape")

DFDIST = append_col_with_grp_index(DFDIST, ["shape_semantic_same", "task_kind_12"], "same_shape|task_kind_12")


In [ ]:
import seaborn as sns
for y in ["dist_mean", "dist_norm", "dist_yue_diff"]:
    # sns.relplot(data=DFDIST, x="time_bin", y=y, hue="same_shape|task_kind_12", kind="line", errorbar=("ci", 68))
    sns.relplot(data=DFDIST, x="time_bin", y=y, hue="same-task|shape", kind="line", errorbar=("ci", 68))
    # fig = sns.relplot(data=DFDIST, x="time_bin", y=y, kind="line", hue="shape_novel_12")
    # fig = sns.relplot(data=DFDIST, x="time_bin", y=y, kind="line", hue="seqc_0_shape_12", col="shape_novel_12", legend=False, alpha=0.1)


In [ ]:
import seaborn as sns
sns.catplot(data=dfres, x = "bregion", hue="shape_task_same", y="dist_yue_diff", kind="bar", aspect=2)


In [ ]:
# Function...
from neuralmonkey.scripts.analy_euclidian_chars_sp import euclidian_time_resolved


SAVEDIR = "/tmp/TEST_TIME_RESOLV"
os.makedirs(SAVEDIR, exist_ok=True)

bregion = "preSMA_a"    
prune_version = "sp_char_0"
subspace_projection = None
# subspace_projection = "shape_prims_single"
NPCS_KEEP = 10
raw_subtract_mean_each_timepoint = False
remove_drift = False
twind_analy = (-0.15, 0.2)
tbin_dur = 0.1
tbin_slide = 0.02


euclidian_time_resolved(animal, date, DFallpa, bregion, prune_version, remove_drift, savedir, twind_analy,
                            tbin_dur, tbin_slide, 
                            subspace_projection, NPCS_KEEP, 
                            n_min_trials_per_shape = 5, raw_subtract_mean_each_timepoint=raw_subtract_mean_each_timepoint)


In [ ]:
if True:
    # testing by hand
    # remove = [1047, 1048, 1051, 1052, 1056, 1058]
    # remove = [1048, 1051, 1052, 1056, 1058, 1059, 1062]
    remove = [1052, 1056, 1057, 1062, 1073, 1075]
    PA = DFallpa["pa"].values[2]
    hack_prune_to_these_chans = [c for c in PA.Chans if c not in remove]
else:
    hack_prune_to_these_chans = None

# hack_prune_to_these_chans = [1049, 1053, 1054, 1057]
# hack_prune_to_these_chans = [1043, 1044, 1047, 1053, 1054, 1057]


In [ ]:
from neuralmonkey.scripts.analy_euclidian_chars_sp import euclidian_time_resolved
import os
# prune_version = "sp_char_0"
# prune_version = None
n_min_trials_per_shape = 5
raw_subtract_mean_each_timepoint = False


SAVEDIR_ANALYSIS = "/tmp/TEST_TIME_RESOLV"
os.makedirs(SAVEDIR_ANALYSIS, exist_ok=True)

NPCS_KEEP = 10
raw_subtract_mean_each_timepoint = False

twind_analy = (-0.35, 0.5)
tbin_dur = 0.1
tbin_slide = 0.02

# # for bregion in DFallpa["bregion"].unique().tolist():
# for bregion in ["PMv_m"]:
#     # for prune_version in ["sp_char_0", "pig_char_0"]:
#     for prune_version in ["sp_char", "pig_char", "sp_char_0"]:
#         # for subspace_projection in [None, "pca", "shape_prims_single", "shape_all"]:
#         for subspace_projection in [None]:
#             for remove_drift in [False]:
#                 for remove_singleprims_unstable in [True, False]:

#                     SAVEDIR = f"{SAVEDIR_ANALYSIS}/{bregion}-prune={prune_version}-ss={subspace_projection}-nodrift={remove_drift}-SpUnstable={remove_singleprims_unstable}-HACK-{hack_prune_to_these_chans is not None}"
#                     os.makedirs(SAVEDIR, exist_ok=True)
#                     print(SAVEDIR)


#                     euclidian_time_resolved(animal, date, DFallpa, bregion, prune_version, remove_drift, SAVEDIR, twind_analy,
#                                                 tbin_dur, tbin_slide, 
#                                                 subspace_projection, NPCS_KEEP, 
#                                                 n_min_trials_per_shape = 5, 
#                                                 raw_subtract_mean_each_timepoint=raw_subtract_mean_each_timepoint,
#                                                 hack_prune_to_these_chans=hack_prune_to_these_chans,
#                                                 remove_singleprims_unstable=remove_singleprims_unstable)

#                     assert False

# Compact version of above
from neuralmonkey.scripts.analy_euclidian_chars_sp import euclidian_time_resolved_wrapper
euclidian_time_resolved_wrapper(animal, date, DFallpa, "/tmp/TEST")

In [ ]:
n_min_trials_per_shape = 4
raw_subtract_mean_each_timepoint = False

NPCS_KEEP = 6

twind_analy = (-0.4, 0.5)
tbin_dur = 0.1
tbin_slide = 0.02

for bregion in DFallpa["bregion"].unique().tolist():
    for prune_version in ["sp_char_0", "pig_char_0", "sp_char", "pig_char_1plus"]:
        for subspace_projection in ["task_shape_si"]:
            for remove_drift in [False]:
                for raw_subtract_mean_each_timepoint in [False]:
                    for remove_singleprims_unstable in [True]:
                        SAVEDIR = f"{SAVEDIR_ANALYSIS}/{bregion}-prune={prune_version}-ss={subspace_projection}-nodrift={remove_drift}-SpUnstable={remove_singleprims_unstable}-subtrmean={raw_subtract_mean_each_timepoint}"
                        os.makedirs(SAVEDIR, exist_ok=True)
                        print("SAVING AT ... ", SAVEDIR)
                        euclidian_time_resolved(animal, date, DFallpa, bregion, prune_version, remove_drift, SAVEDIR, twind_analy,
                                                    tbin_dur, tbin_slide, 
                                                    subspace_projection, NPCS_KEEP, 
                                                    n_min_trials_per_shape = n_min_trials_per_shape, raw_subtract_mean_each_timepoint=raw_subtract_mean_each_timepoint,
                                                    remove_singleprims_unstable=remove_singleprims_unstable)


# [3] Samp, Go, reach onset

In [ ]:
import os
from neuralmonkey.scripts.analy_euclidian_chars_sp import euclidian_time_resolved

SAVEDIR_ANALYSIS = "/tmp/SAMP"
os.makedirs(SAVEDIR_ANALYSIS, exist_ok=True)


In [ ]:
DFallpa

In [ ]:
n_min_trials_per_shape = 4
raw_subtract_mean_each_timepoint = False
tbin_dur = 0.1
tbin_slide = 0.02
twind_analy = (-1, 1)
NPCS_KEEP = 7
events_keep = ["03_samp", "05_first_raise"]

HACK = True
for i, row in DFallpa.iterrows():
    bregion = row["bregion"]
    which_level = row["which_level"]
    event = row["event"]
    PA = row["pa"]

    if HACK and bregion not in ["PMv"]:
        continue

    if event in events_keep:
        for prune_version in ["sp_char"]:
            for subspace_projection in [None, "pca", "task_shape"]: # NOTE: shape_prims_single not great, you lose some part of preSMA context-dependence...
                for remove_drift in [False]:
                    for raw_subtract_mean_each_timepoint in [False]:
                        for remove_singleprims_unstable in [False, True]:
                            SAVEDIR = f"{SAVEDIR_ANALYSIS}/{which_level}-{bregion}-{event}-prune={prune_version}-ss={subspace_projection}-nodrift={remove_drift}-SpUnstable={remove_singleprims_unstable}-subtrmean={raw_subtract_mean_each_timepoint}"
                            os.makedirs(SAVEDIR, exist_ok=True)
                            print("SAVING AT ... ", SAVEDIR)
                            euclidian_time_resolved(animal, date, PA, which_level, prune_version, remove_drift, SAVEDIR, twind_analy,
                                                        tbin_dur, tbin_slide, 
                                                        subspace_projection, NPCS_KEEP, 
                                                        n_min_trials_per_shape = n_min_trials_per_shape, raw_subtract_mean_each_timepoint=raw_subtract_mean_each_timepoint,
                                                        remove_singleprims_unstable=remove_singleprims_unstable)


# Time warping based on behavior

In [ ]:
# For each trial, get beh stroke
date

# Checking drift of FR

In [ ]:
# Plot for each trial.
from neuralmonkey.scripts.analy_euclidian_chars_sp import plot_heatmap_firing_rates_all
plot_heatmap_firing_rates_all(PA, savedir)


### Sanity check, change over day?


In [ ]:
savedir = f"{SAVEDIR_ANALYSIS}/drift"
os.makedirs(savedir, exist_ok=True)

##### For each region, plot sm fr across trials and time

In [ ]:
# Loop over all bregions
assert False, "run for both chans and PCs"
for i, row in DFallpa.iterrows():
    PAthis = row["pa"]
    bregion = row["bregion"]

    dur = 0.1
    slide = 0.02
    PAthis = PAthis.agg_by_time_windows_binned(dur, slide)


    from neuralmonkey.neuralplots.population import heatmapwrapper_stratified_each_neuron
    fig = heatmapwrapper_stratified_each_neuron(PA, "task_kind")
    assert False
    assert False
    savefig(fig, f"{savedir}/{bregion}.png")
    plt.close("all")
    



In [ ]:
import os
SAVEDIR = f"/tmp/CHAR_HEATMAPS"
os.makedirs(SAVEDIR, exist_ok=True)

In [ ]:
# (1) First, dim reduction
superv_dpca_var = superv_dpca_params['superv_dpca_var']
superv_dpca_vars_group = superv_dpca_params['superv_dpca_vars_group']
superv_dpca_filtdict = superv_dpca_params['superv_dpca_filtdict']


dim_red_method = "pca"
twind_analy = (-0.15, 0.3)
tbin_dur = 0.1
tbin_slice = 0.01
NPCS_KEEP = 10
_, PAredu = PA.dataextract_dimred_wrapper("traj", dim_red_method, savedir, 
                                twind_analy, tbin_dur=tbin_dur, tbin_slide=tbin_slice, 
                                NPCS_KEEP = NPCS_KEEP,
                                dpca_var = None, dpca_vars_group = None, dpca_filtdict=None, 
                                dpca_proj_twind = twind_analy, 
                                raw_subtract_mean_each_timepoint=raw_subtract_mean_each_timepoint,
                                umap_n_components=None, umap_n_neighbors=None)


In [ ]:
PAredu.Times

In [ ]:
##### Script for heatmaps

In [ ]:
SAVEDIR

In [ ]:
date

In [ ]:
from neuralmonkey.scripts.analy_euclidian_chars_sp import params_subspace_projection
from neuralmonkey.scripts.analy_euclidian_chars_sp import plot_heatmap_firing_rates_all, plot_heatmap_firing_rates_all_wrapper

plot_heatmap_firing_rates_all_wrapper(DFallpa, SAVEDIR, animal, date)


### Accounting for drift -- exclude cases with very different PSTH.

In [ ]:
PA = DFallpa["pa"].values[2]

In [ ]:
from neuralmonkey.scripts.analy_euclidian_chars_sp import preprocess_clean_stable_single_prims_frate
savedir = "/tmp/test"
os.makedirs(savedir, exist_ok=True)
preprocess_clean_stable_single_prims_frate(PA, savedir=savedir)

##### Re-apply filter based on dirty sites, being more strict about drift

In [ ]:
# For each grouping, get the meanPSTH, and c

# MULT DATA - euclidian_time_resolved

In [ ]:
import pandas as pd

In [ ]:
from neuralmonkey.classes.session import _REGIONS_IN_ORDER, _REGIONS_IN_ORDER_COMBINED

import pandas as pd



In [ ]:
from neuralmonkey.scripts.analy_euclidian_chars_sp_MULT import  load_all_dates
# Diego (trial, all)
LIST_ANIMAL_DATE_COMB = [
    ("Diego", 231122, True),
    ("Diego", 231128, True),
    ("Diego", 231129, True),
    ("Diego", 231201, True),
    ("Diego", 231204, True),
    ("Diego", 231205, True),
    ("Diego", 231211, True),
    ("Diego", 231213, True),
]
which_level = "trial"
savedir_method_old = False

DFDIST = load_all_dates(LIST_ANIMAL_DATE_COMB, which_level, savedir_method_old)

In [ ]:
DFDIST

In [ ]:
# NOTE: For plots, see analy_euclidian_chars_sp_MULT.py

In [ ]:
from neuralmonkey.scripts.analy_euclidian_chars_sp_MULT import  plot_scalar_all
from pythonlib.tools.pandastools import aggregGeneral
from pythonlib.tools.pandastools import grouping_append_and_return_inner_items_good
from pythonlib.tools.pandastools import plot_45scatter_means_flexible_grouping
from pythonlib.tools.plottools import savefig
import os
import matplotlib.pyplot as plt


In [ ]:
twind_scalar = [-0.3, 0.1]
DFDISTthis = DFDIST[(DFDIST["time_bin"]>=twind_scalar[0]) & (DFDIST["time_bin"]<=twind_scalar[1])].reset_index(drop=True)

# Agg, averaging over time
DFTHISscal = aggregGeneral(DFDISTthis, ["animal", "date", "combine_areas", "event", "bregion", "metaparams", "same-task|shape", "prune_version", "subspace_projection", "remove_drift", "raw_subtract_mean_each_timepoint", 
                                "remove_singleprims_unstable"], values=["dist_mean", "dist_norm", "dist_yue_diff"])



In [ ]:
map_event_to_twind = {
    "03_samp":[0.2, 1.0],
    "05_first_raise":[-0.5,  0],
}

list_df = []
for event, twind_scalar in map_event_to_twind.items():
    dfthis = DFDIST[DFDIST["event"] == event]
    dfthis_sub = dfthis[(dfthis["time_bin"]>=twind_scalar[0]-0.001) & (dfthis["time_bin"]<=twind_scalar[1]+0.001)].reset_index(drop=True)
    list_df.append(dfthis_sub)
DFDISTthis = pd.concat(list_df).reset_index(drop=True)

# Agg, averaging over time
DFTHISscal = aggregGeneral(DFDISTthis, ["animal", "date", "combine_areas", "event", "bregion", "metaparams", "same-task|shape", "prune_version", "subspace_projection", "remove_drift", "raw_subtract_mean_each_timepoint", 
                                "remove_singleprims_unstable"], values=["dist_mean", "dist_norm", "dist_yue_diff"])


In [ ]:
from pythonlib.tools.expttools import writeDictToTxt
writeDictToTxt(map_event_to_twind, f"{savedir}/twind_params.txt")


In [ ]:
# Final scalar plot
DFTHISscal

In [ ]:
savedir = "/tmp/SCALAR"
os.makedirs(savedir, exist_ok=True)
grpdict = grouping_append_and_return_inner_items_good(DFTHISscal, ["event", "subspace_projection", "remove_drift", "raw_subtract_mean_each_timepoint", 
                                                                "remove_singleprims_unstable"])
for (event, subspace_projection, remove_drift, raw_subtract_mean_each_timepoint, remove_singleprims_unstable), inds in grpdict.items():
    dfthis = DFTHISscal.iloc[inds].reset_index(drop=True)

    _, fig = plot_45scatter_means_flexible_grouping(dfthis, "same-task|shape", "0|1", "1|0", 
                                        "prune_version", "dist_yue_diff", "bregion", 
                                        True, shareaxes=True, SIZE=4);
    savefig(fig, f"{savedir}/EVENT={event}-ss={subspace_projection}-rmvdrift={remove_drift}-subtrmean={raw_subtract_mean_each_timepoint}-rmvunstable={remove_singleprims_unstable}.pdf")
    plt.close("all")


##### Convert to scalar

In [ ]:
twind_scalar = [-0.3, 0.1]
from neuralmonkey.scripts.analy_euclidian_chars_sp_MULT import plot_scalar_all
plot_scalar_all(DFDIST, "/tmp", twind_scalar)

### Combine all kinds of pairwise comparisons in the same plot

In [ ]:
from pythonlib.tools.plottools import savefig
savedir = "/tmp"

In [ ]:
from pythonlib.tools.pandastools import grouping_append_and_return_inner_items_good
grpdict = grouping_append_and_return_inner_items_good(DFTHISscal, ["subspace_projection", "remove_drift", "raw_subtract_mean_each_timepoint", 
                                                                   "remove_singleprims_unstable"])
for (subspace_projection, remove_drift, raw_subtract_mean_each_timepoint, remove_singleprims_unstable), inds in grpdict.items():
    dfthis = DFTHISscal.iloc[inds].reset_index(drop=True)

    from pythonlib.tools.pandastools import plot_45scatter_means_flexible_grouping
    _, fig = plot_45scatter_means_flexible_grouping(dfthis, "same-task|shape", "0|1", "1|0", 
                                        "metaparams", "dist_yue_diff", "bregion", 
                                        True, shareaxes=True, SIZE=4);    
    savefig(fig, f"{savedir}/ss={subspace_projection}-rmvdrift={remove_drift}-subtrmean={raw_subtract_mean_each_timepoint}-rmvunstable={remove_singleprims_unstable}")
    plt.close("all)")

# Example figures (for paper)

In [ ]:
from neuralmonkey.scripts.analy_euclidian_chars_sp import behstrokes_preprocess_assign_col_bad_strokes, preprocess_pa
import numpy as np

In [ ]:

behstrokes_preprocess_assign_col_bad_strokes(DFallpa, animal, date)


### [Code] plot_heatmap_firing_rates_all_wrapper

In [ ]:
bregion = "PMv"
PA = extract_single_pa(DFallpa, bregion, which_level="stroke", event="00_stroke")
print(PA.X.shape)

prune_version = "sp_char_0"
n_min_trials_per_shape = 4
shape_var = "shape_semantic_grp"
plot_drawings = False
# twind_analy = (-0.4, 0.5)
twind_analy = (-0.35, 0.3)
tbin_dur=0.1
tbin_slide=0.01

NPCS_KEEP = 10
subspace_projection = "pca"
# NPCS_KEEP = None
# subspace_projection = None
raw_subtract_mean_each_timepoint = False

savedir = "/tmp"
PA = preprocess_pa(animal, date, PA, savedir, prune_version, 
                    n_min_trials_per_shape=n_min_trials_per_shape, shape_var=shape_var, plot_drawings=plot_drawings,
                    remove_chans_fr_drift=False,
                    subspace_projection=subspace_projection, 
                        twind_analy=twind_analy, tbin_dur=tbin_dur, tbin_slide=tbin_slide, NPCS_KEEP=NPCS_KEEP,
                        raw_subtract_mean_each_timepoint=raw_subtract_mean_each_timepoint,
                        remove_singleprims_unstable=True)



In [ ]:
from neuralmonkey.scripts.analy_euclidian_chars_sp import plot_heatmap_firing_rates_all_wrapper
import os
SAVEDIR_ANALYSIS = "/tmp/HEATMAP_WRAPPER"
os.makedirs(SAVEDIR_ANALYSIS, exist_ok=True)
plot_heatmap_firing_rates_all_wrapper(DFallpa, SAVEDIR_ANALYSIS, animal, date, 
                                      DEBUG_skip_drawings=True, DEBUG_bregion="PMv",
                                      DEBUG_subspace_projection="pca_proj")

In [ ]:
from neuralmonkey.scripts.analy_euclidian_chars_sp import behstrokes_preprocess_assign_col_bad_strokes, preprocess_pa
behstrokes_preprocess_assign_col_bad_strokes(DFallpa, animal, date)

In [ ]:
from neuralmonkey.classes.session import load_session_helper, load_mult_session_helper
MS = load_mult_session_helper(date, animal)   



In [ ]:
from neuralmonkey.scripts.analy_euclidian_chars_sp import beh_plot_event_timing_stroke
savedir = "/tmp"
PA = DFallpa.iloc[0]["pa"]
beh_plot_event_timing_stroke(PA, animal, date, savedir, MS=MS)

In [ ]:
import os

### [MULT DAYS] for: euclidian_time_resolved_fast_shuffled


In [ ]:
from neuralmonkey.scripts.analy_shape_invariance_all_plots_SP import euclidian_time_resolved_fast_shuffled_mult_scatter_plots

In [ ]:
##### (1) Original plots, still good.

In [ ]:
if False:
    # Original paper
    analysis_kind = "char_sp_00_stroke"
    # analysis_kind = "char_sp_05_first_raise"
    # analysis_kind = "char_sp_04_go_cue"
else:
    # Revision
    # analysis_kind = "char_sp_00_stroke_revision_noregr"
    analysis_kind = "char_sp_00_stroke_revision_regr"
    analysis_kind = "char_sp_00_stroke_revision_regr_seman"
    
prune_min_n_trials = None
DFDISTS, DFDISTS_AGG = euclidian_time_resolved_fast_shuffled_mult_scatter_plots(analysis_kind, just_return_df=True, 
                                                                                               DO_FURTHER_POSTPROCESSING=False,
                                                                                               prune_min_n_trials=prune_min_n_trials)

##### Random stuff

In [ ]:
path = "/lemur2/lucas/analyses/recordings/main/euclidian_char_sp/EUCL_QUICK_SHUFFLE_revision-VAR_SHAPE=shape_semantic_grp/Diego-231220-combine=True-wl=stroke-00_stroke-regrhack=True/list_dfdist.pkl"
# path = "/lemur2/lucas/analyses/recordings/main/euclidian_char_sp/EUCL_QUICK_SHUFFLE_revision/Diego-231220-combine=True-wl=stroke-00_stroke-regrhack=True/list_dfdist.pkl"
import pickle
with open(path, "rb") as f:
    list_dfdist = pickle.load(f)

In [ ]:
from pythonlib.tools.pandastools import grouping_print_n_samples, grouping_plot_n_samples_conjunction_heatmap
from pythonlib.tools.plottools import savefig
savedir = "/tmp"
grpdict = grouping_append_and_return_inner_items_good(DFDISTS, ["animal", "date", "prune_version"])
print(len(grpdict))
for grp, inds in grpdict.items():
    print(grp)
    dfdists = DFDISTS.iloc[inds].reset_index(drop=True)
    # grouping_print_n_samples(dfdists, ["animal", "date", "metaparams", "same-shape_semantic_grp|task_kind", "shape_semantic_grp_1", "shape_semantic_grp_2"])
    # grouping_print_n_samples(dfdists, ["same-shape_semantic_grp|task_kind", "shape_semantic_grp_1", "shape_semantic_grp_2", "animal", "date"])
    # asds
    fig = grouping_plot_n_samples_conjunction_heatmap(dfdists, "shape_semantic_grp_1", "shape_semantic_grp_2", 
                                                ["same-shape_semantic_grp|task_kind"], annotate_heatmap=False,
                                                FIGSIZE=4, n_columns=4)
    savefig(fig, f"{savedir}/allpairs_counts-{grp}.pdf")
    plt.close("all")


### Good, for revision (all stroke indices)


In [ ]:

if False:
    import time
    nhrs = 4
    nsec = nhrs * 60 * 60
    print("sleeping this many sec: ", nsec)
    time.sleep(nsec)

# Just plot a subset, or else will take forever
# list_metaparams_plot_each_pair = [
#     "sp_char_1plus|task_shape_si|(-0.8, 0.3)|00_stroke|(-0.5, -0.05)"
#     ]
list_metaparams_plot_each_pair = [
    # "sp_char_1plus|task_shape|(-0.8, 0.3)|00_stroke|(-0.5, -0.05)"
    "sp_char_0|task_shape|(-0.8, 0.3)|00_stroke|(-0.35, -0.05)",
    "sp_char_1plus|task_shape|(-0.8, 0.3)|00_stroke|(-0.35, -0.05)",
    "sp_pig_1plus|task_shape|(-0.8, 0.3)|00_stroke|(-0.35, -0.05)",
    ]

for PLOT_EACH_PAIR in [False, True]: # First False to run thru quickly
    # for analysis_kind in ["char_sp_00_stroke_revision_regr", "char_sp_00_stroke_revision_noregr"]:
    # for analysis_kind in ["char_sp_00_stroke_revision_regr"]:
    for analysis_kind in ["char_sp_00_stroke_revision_regr_seman", "char_sp_00_stroke_revision_noregr_seman"]:
        # for DO_FURTHER_POSTPROCESSING in [True, False]:
        for DO_FURTHER_POSTPROCESSING in [True]: # False doesnt do much
        # for DO_FURTHER_POSTPROCESSING in [False]: # False is quicker
            # for prune_min_n_trials in [None, 4]:
            for prune_min_n_trials in [None]:

                # Only plot each pair one time
                if PLOT_EACH_PAIR:
                    a = (prune_min_n_trials is None) and (DO_FURTHER_POSTPROCESSING is True)
                    if not a:
                        continue
                euclidian_time_resolved_fast_shuffled_mult_scatter_plots(analysis_kind, 
                                                                        DO_FURTHER_POSTPROCESSING=DO_FURTHER_POSTPROCESSING,
                                                                        PLOT_EACH_PAIR=PLOT_EACH_PAIR, 
                                                                        prune_min_n_trials=prune_min_n_trials,
                                                                        list_metaparams_plot_each_pair=list_metaparams_plot_each_pair)


In [ ]:
### Debug -- Try replotting after removing specific shapes.
var_effect = "shape_semantic"
var_value = "dist_yue_diff"
var_same_same = f"same-{var_effect}|task_kind"

from pythonlib.tools.pandastools import plot_45scatter_means_flexible_grouping, grouping_append_and_return_inner_items_good

### Original
if False:
    grp_vars = ["which_level", "prune_version", "subspace|twind", "event"]
    grpdict_dat = grouping_append_and_return_inner_items_good(DFDISTS_AGG, grp_vars)


    dfthis_dat = DFDISTS
    _, fig = plot_45scatter_means_flexible_grouping(dfthis_dat, var_same_same, "1|0", "0|1", "metaparams", 
                                        var_value, "bregion", True, shareaxes=True, SIZE=3.5)

### Exclude line
# DFDISTS = DFDISTS[[x[0]!="line-UL-UL" for x in DFDISTS["labels_1"]]].reset_index(drop=True)
# dfthis_dat = DFDISTS[["ARC" in x[0] for x in DFDISTS["labels_1"]]].reset_index(drop=True)
# dfthis_dat = DFDISTS[["line" not in x[0] for x in DFDISTS["labels_1"]]].reset_index(drop=True)
from neuralmonkey.scripts.analy_shape_invariance_all_plots_SP import _remove_shape
list_sh_remove = ["Lcentered-DR-DR", "V-DD-DD"]
DFDISTS_THIS, DFDISTS_THIS_AGG = _remove_shape(DFDISTS, list_sh_remove, var_same_same)

In [ ]:
# Plot scatter, stats
var_datapt = None
savedir = "/tmp/STATS"
os.makedirs(savedir, exist_ok=True)

# (3) Specific subspace projections.
list_subspace = ["task_shape"]
prune_version = "sp_char_1plus"

dfdists = DFDISTS_THIS[DFDISTS_THIS["prune_version"] == prune_version].reset_index(drop=True)
assert len(dfdists)>0

# Postprocess
dfdists["labels_pair_unique"] = [tuple(sorted((row["labels_1"], row["labels_2"]))) for _, row in dfdists.iterrows()]
dfdists["shapes_pair_unique"] = [tuple(sorted((row["labels_1"][0], row["labels_2"][0]))) for _, row in dfdists.iterrows()]
dfdists = _euclidianshuff_stats_linear_load_mult_dates_postprocess(dfdists)

### Plot
DFSTATS_2BR = _euclidianshuff_stats_linear_2br_scatter_wrapper(dfdists, var_same_same, var_datapt, savedir, 
                                                               plot_heatmap_counts=False, plot_catplots=False)


In [ ]:

# DFDISTS = DFDISTS[[x[0]!="line-UL-UL" for x in DFDISTS["labels_1"]]].reset_index(drop=True)
# dfthis_dat = DFDISTS[["ARC" in x[0] for x in DFDISTS["labels_1"]]].reset_index(drop=True)
for shape in ["line", "ARC", "ZZ", "circle", "Lcentered"]:
    print(shape)
    dfthis_dat = DFDISTS[[shape in x[0] for x in DFDISTS["labels_1"]]].reset_index(drop=True)

    _, fig = plot_45scatter_means_flexible_grouping(dfthis_dat, var_same_same, "1|0", "0|1", "metaparams", 
                                        var_value, "bregion", True, shareaxes=True, SIZE=3.5)


In [ ]:
### Debug, prune to remove cases with too low n datapts

In [ ]:
nmin = 5

grpdict = grouping_append_and_return_inner_items_good(DFDISTS, ["metaparams", "animal", "date"])
list_dfdist =[]
for grp, inds in grpdict.items():
    dfdists = DFDISTS.iloc[inds].reset_index(drop=True)

    a = dfdists.loc[:, ["shape_semantic_grp_1", "n1", "task_kind_1"]].values.tolist()
    b = dfdists.loc[:, ["shape_semantic_grp_2", "n2", "task_kind_2"]].values.tolist()
    dftmp = pd.DataFrame(a+b, columns=[var_effect, "n", "task_kind"])

    if False:
        from pythonlib.tools.pandastools import grouping_print_n_samples
        grouping_print_n_samples(dftmp, ["shape_semantic_grp", "n", "task_kind"])

    # The pool of shapes to ignore
    shapes_ignore = dftmp[dftmp["n"] < nmin]["shape_semantic_grp"].unique().tolist()

    # Keep just shapes that are not ignored
    a = dfdists[f"{var_effect}_1"].isin(shapes_ignore)
    b = dfdists[f"{var_effect}_2"].isin(shapes_ignore)

    dfdists_new = dfdists[~(a | b)].reset_index(drop=True)

    # Store
    list_dfdist.append(dfdists_new)

    # print(grp, "       |       ", len(dfdists), " ---> ", len(dfdists_new), f"[{len(dfdists_new)/len(dfdists):.2f}]")

DFDISTS = pd.concat(list_dfdist).reset_index(drop=True)


In [ ]:
# Final summary plot
dfthis_dat = DFDISTS[DFDISTS["metaparams"] == "sp_char_1plus|task_shape_si|(-0.8, 0.3)|00_stroke|(-0.5, -0.05)"].reset_index(drop=True)


In [ ]:


a = dfthis_dat["n1"]<nmin
b = dfthis_dat["n2"]<nmin
print(sum(a), sum(b), len(a))

dfthis_dat = dfthis_dat[(dfthis_dat["n1"]>=nmin) & (dfthis_dat["n2"]>=nmin)].reset_index(drop=True)

from pythonlib.tools.pandastools import plot_45scatter_means_flexible_grouping, grouping_append_and_return_inner_items_good
var_same_same = "same-shape_semantic_grp|task_kind"
var_value = "dist_yue_diff"
_, fig = plot_45scatter_means_flexible_grouping(dfthis_dat, var_same_same, "1|0", "0|1", "metaparams", 
                                    var_value, "bregion", True, shareaxes=True, SIZE=3.5)


##### Find outlier points

In [ ]:
a = DFDISTS["bregion"] == "PMv"
b = DFDISTS["metaparams"] == 'sp_char_1plus|task_shape_si|(-0.8, 0.3)|00_stroke|(-0.3, 0.0)'
c = DFDISTS["same-shape_semantic|task_kind"] == "1|0"
dfdists = DFDISTS[a & b & c]

dfdists["dist_yue_diff"].hist(bins=30)

dfdists[dfdists["dist_yue_diff"]>0.2]


In [ ]:
# Plot distribution for each shapes
from pythonlib.tools.pandastools import aggregGeneral
from neuralmonkey.analyses.euclidian_distance import dfdist_summary_plots_wrapper

shapes = sorted(set(DFDISTS["shape_semantic_1"].tolist() + DFDISTS["shape_semantic_2"].tolist()))
for sh in shapes:
    dfdists = DFDISTS[(DFDISTS["shape_semantic_1"]==sh) | (DFDISTS["shape_semantic_2"]==sh)].reset_index(drop=True)
    dfdists_agg = aggregGeneral(dfdists, ["animal", "date", "metaparams", "bregion", "prune_version", 
                                            "which_level", "event", "subspace|twind", var_same_same],
                                ["dist_mean", "dist_norm", "dist_yue_diff", "DIST_50", "DIST_98"], nonnumercols="all")
        
    savedir = f"/tmp/TESTING/shape={sh}"
    os.makedirs(savedir, exist_ok=True)
    do_catplots = False
    do_quick = True
    dfdist_summary_plots_wrapper(dfdists, dfdists_agg, var_effect, var_other, SAVEDIR=savedir,
                                PLOT_EACH_PAIR=False,  
                                list_metaparams_plot_each_pair=list_metaparams_plot_each_pair,
                                do_catplots=do_catplots, do_quick=do_quick)
    plt.close("all")

##### (2) Stats, and scatter plots. Good.

In [5]:
from neuralmonkey.scripts.analy_shape_invariance_all_plots_SP import _euclidianshuff_stats_linear_load_mult_dates_postprocess, euclidianshuff_stats_linear_plot_wrapper, euclidian_time_resolved_fast_shuffled_mult_reload, euclidian_time_resolved_fast_shuffled_mult_scatter_plots


In [8]:
from neuralmonkey.scripts.analy_shape_invariance_all_plots_SP import euclidian_time_resolved_fast_shuffled_mult_reload, _euclidian_time_resolved_fast_shuffled_mult_scatter_plots_params
from pythonlib.tools.pandastools import convert_var_to_categorical
from pythonlib.tools.plottools import savefig
from neuralmonkey.scripts.analy_shape_invariance_all_plots_SP import euclidianshuff_stats_linear_load, euclidianshuff_stats_linear_plot_wrapper, euclidian_time_resolved_fast_shuffled_mult_reload, euclidian_time_resolved_fast_shuffled_mult_scatter_plots
from pythonlib.tools.pandastools import convert_var_to_categorical
from pythonlib.tools.plottools import savefig
from neuralmonkey.neuralplots.brainschematic import datamod_reorder_by_bregion
from pythonlib.tools.pandastools import grouping_append_and_return_inner_items_good
from pythonlib.tools.statstools import signrank_wilcoxon_from_df
import numpy as np
import pandas as pd
from neuralmonkey.scripts.analy_shape_invariance_all_plots_SP import euclidianshuff_stats_linear_load, euclidianshuff_stats_linear_plot_wrapper, euclidian_time_resolved_fast_shuffled_mult_reload, euclidian_time_resolved_fast_shuffled_mult_scatter_plots
import os
from neuralmonkey.scripts.analy_shape_invariance_all_plots_SP import _euclidianshuff_stats_linear_load_mult_dates_postprocess, euclidianshuff_stats_linear_load, euclidianshuff_stats_linear_plot_wrapper, euclidian_time_resolved_fast_shuffled_mult_reload, euclidian_time_resolved_fast_shuffled_mult_scatter_plots

subsample_plot_each_iter_stats = True

if False:
    # Original submission params
    merge_pancho_ss_twinds=False
    plot_coeff = False
    # analysis_kind = "char_sp"
    # analysis_kind = "char_sp_05_first_raise"
    # analysis_kind = "char_sp_04_go_cue"
    analysis_kind = "char_sp_00_stroke"
    do_each_date = False
    var_effect = "shape_semantic_grp"
    # for pruning.
    list_prune_version = ["sp_char_0", "pig_char_0"]
    list_subspace = ["task_shape", "task_shape_si"]
    # list_subspace = ["task_shape_si", "shape_all"]
    # list_prune_version = ["sp_char_0"]
    # list_subspace = ["task_shape_si"]
else:
    # Revision params
    # analysis_kind = "char_sp_00_stroke_revision_regr_seman"
    do_each_date = False
    var_effect = "shape_semantic"
    # list_prune_version = ["sp_char_0", "sp_char_1plus", "sp_pig_1plus", "pig0_char_1plus"]
    list_prune_version = ["sp_char_0", "sp_char_1plus", "sp_pig_1plus"]
    list_subspace = ["task_shape", "task_shape_si"]
    # list_subspace = ["task_shape_si"]
    is_revision=True
    consolidate_diego_shapes = True
    
# PRUNE_SHAPE = False
old_run_number = None
for PRUNE_SHAPE in [False]:
    ## Revision -- loop 
    for analysis_kind in ["char_sp_00_stroke_revision_noregr_seman", "char_sp_00_stroke_revision_regr_seman"]:
        do_each_date = False
        var_effect = "shape_semantic"
        # list_prune_version = ["sp_char_0", "sp_char_1plus", "sp_pig_1plus", "pig0_char_1plus"]
        list_prune_version = ["sp_char_0", "sp_char_1plus", "sp_pig_1plus"]
        # list_subspace = ["task_shape_si", "task_shape"]
        list_subspace = ["task_shape"]
        is_revision=True
        consolidate_diego_shapes = True

        for animal in ["Diego", "Pancho"]:

            # if PRUNE_SHAPE==False and analysis_kind=="char_sp_00_stroke_revision_regr_seman" and animal=="Diego":
            #     continue

            for var_other in ["task_kind"]:

                list_date = _euclidian_time_resolved_fast_shuffled_mult_scatter_plots_params(analysis_kind, animal, var_other)
                var_same_same = f"same-{var_effect}|{var_other}"

                if do_each_date:
                    for date in list_date:
                        dfdists, SAVEDIR_PLOTS = euclidianshuff_stats_linear_load(animal, date, var_other)

                        ### ALL PLOTS
                        euclidianshuff_stats_linear_plot_wrapper(dfdists, SAVEDIR_PLOTS, var_other)

                from neuralmonkey.scripts.analy_shape_invariance_all_plots_SP import euclidianshuff_stats_linear_load_mult_dates
                DFDISTS_ALL = euclidianshuff_stats_linear_load_mult_dates(animal, list_date, var_other, analysis_kind, 
                                                                        old_run_number=old_run_number)

                for prune_version in list_prune_version:
                    if old_run_number is None:
                        if is_revision:
                            SAVEDIR_PLOTS = f"/lemur2/lucas/analyses/recordings/main/euclidian_char_sp/EUCL_QUICK_SHUFFLE_revision-VAR_SHAPE={var_effect}/MULT/stats_linear_model/{analysis_kind}/{animal}-{'|'.join([str(d) for d in list_date])}-var_other={var_other}-prune={prune_version}-ss={list_subspace}-prunesh={PRUNE_SHAPE}"
                            os.makedirs(SAVEDIR_PLOTS, exist_ok=True)
                        else:
                            SAVEDIR_PLOTS = f"/lemur2/lucas/analyses/recordings/main/euclidian_char_sp/EUCL_QUICK_SHUFFLE/MULT/stats_linear_model/{analysis_kind}/{animal}-{'|'.join([str(d) for d in list_date])}-var_other={var_other}-prune={prune_version}-ss={list_subspace}-prunesh={PRUNE_SHAPE}"
                            os.makedirs(SAVEDIR_PLOTS, exist_ok=True)
                    else:
                        assert is_revision
                        SAVEDIR_PLOTS = f"/lemur2/lucas/analyses/recordings/main/euclidian_char_sp/EUCL_QUICK_SHUFFLE_revision_runs/run{old_run_number}/MULT/stats_linear_model/{analysis_kind}/{animal}-{'|'.join([str(d) for d in list_date])}-var_other={var_other}-prune={prune_version}-ss={list_subspace}-prunesh={PRUNE_SHAPE}"
                        os.makedirs(SAVEDIR_PLOTS, exist_ok=True)

                    # Condition
                    # (1) For now, just plot the ones that you care about (or else takes a long time)
                    DFDISTS = DFDISTS_ALL[DFDISTS_ALL["prune_version"] == prune_version].reset_index(drop=True)
                    if len(DFDISTS)>0:
                        # (2) FIgure if add one day that only had PIG to sp_char_0 analyses
                        # Combine multiple prune versions.
                        # In some cases the day doenst have single prims -- in which case should use PIG (pig_char_0).
                        if False:
                            if animal == "Pancho" and prune_version == "sp_char_0":    
                                prune_extract_append = "pig_char_0"
                                dates_extract_append = [220614]
                            elif animal == "Diego" and prune_version == "sp_char_0":    
                                prune_extract_append = "pig_char_0"
                                dates_extract_append = [231120]
                            else:
                                prune_extract_append = None
                                dates_extract_append = None
                        else:
                            # Revision, no need to add these
                            prune_extract_append = None
                            dates_extract_append = None

                        if prune_extract_append is not None:
                            assert not prune_extract_append == prune_version
                            _df = DFDISTS_ALL[(DFDISTS_ALL["prune_version"] == prune_extract_append) & (DFDISTS_ALL["date"].isin(dates_extract_append))].reset_index(drop=True)
                            # cleanup so that fakes that this is prims_single, or else downstream code fails.
                            _df["prune_version"] = prune_version
                            def _replace_pig_with_sp(x):
                                """
                                eg.
                                x = (ARC-RR, prims_on_grid)	
                                returns:
                                (ARC-RR, prims_single)	
                                """
                                
                                if x[1]=="prims_on_grid":
                                    return (x[0], "prims_single")
                                else:
                                    return (x[0], x[1])
                            _df["labels_1"] = [_replace_pig_with_sp(x) for x in _df["labels_1"]]
                            _df["labels_2"] = [_replace_pig_with_sp(x) for x in _df["labels_2"]]

                            DFDISTS = pd.concat([DFDISTS, _df]).reset_index(drop=True)
                        else:
                            pass
                        
                        # Remove shapes with bad beh
                        if PRUNE_SHAPE:
                            from neuralmonkey.scripts.analy_shape_invariance_all_plots_SP import _remove_shape
                            if animal == "Diego":
                                # list_sh_remove = ["Lcentered-DR-DR", "V-DD-DD"]
                                # list_sh_remove = ["arcdeep-RR-RR", "Lcentered-DL-DL", "Lcentered-DR-DR", "squiggle3-UU-0.0", "V-DD-DD"]
                                list_sh_remove = ["Lcentered-DR-DR"]
                            elif animal == "Pancho":
                                list_sh_remove = ["line-UR-UR"]
                            DFDISTS, _ = _remove_shape(DFDISTS, list_sh_remove, var_same_same)

                        # (3) Specific subspace projections.
                        DFDISTS = DFDISTS[DFDISTS["subspace_projection"].isin(list_subspace)].reset_index(drop=True)
                        assert len(DFDISTS)>0

                        ### Further consolidate Diego shapes
                        if consolidate_diego_shapes:
                            # merge all the shapes
                            from pythonlib.drawmodel.tokens import MAP_SHAPESEM_TO_SHAPESEMGROUP, map_shsem_to_new_shsem
                            from neuralmonkey.analyses.euclidian_distance import dfdist_extract_label_vars_specific
                            from pythonlib.tools.pandastools import aggregGeneral

                            # (1) Rename the labels
                            # DFDISTS["labels_1"].value_counts()
                            DFDISTS["labels_1"] = [x if x[0] not in map_shsem_to_new_shsem else (map_shsem_to_new_shsem[x[0]], x[1]) for x in DFDISTS["labels_1"]]
                            DFDISTS["labels_2"] = [x if x[0] not in map_shsem_to_new_shsem else (map_shsem_to_new_shsem[x[0]], x[1]) for x in DFDISTS["labels_2"]]

                            if True:
                                # (2) This important, becuase for Diego, if merge shapes, someitmes labels_1 is same as labels_2. Need to agg using uinque
                                DFDISTS["labels_pair_unique"] = [tuple(sorted((row["labels_1"], row["labels_2"]))) for _, row in DFDISTS.iterrows()]
                                DFDISTS = aggregGeneral(DFDISTS, ["animal", "date", "bregion", "prune_version", "which_level", "event", "metaparams",
                                                        "subspace_projection", "subspace_projection_fitting_twind", "subspace|twind",
                                                            "twind_scal", "labels_pair_unique"], ["dist_mean", "dist_norm", "dist_yue_diff", "DIST_50", "DIST_98", "n1", "n2"])
                                DFDISTS["labels_1"] = [x[0] for x in DFDISTS["labels_pair_unique"]]
                                DFDISTS["labels_2"] = [x[1] for x in DFDISTS["labels_pair_unique"]]
                            else:
                                # (2) Average the labels if they exist
                                DFDISTS = aggregGeneral(DFDISTS, ["animal", "date", "bregion", "prune_version", "which_level", "event", "metaparams",
                                                        "subspace_projection", "subspace_projection_fitting_twind", "subspace|twind",
                                                            "twind_scal", "labels_1", "labels_2"], ["dist_mean", "dist_norm", "dist_yue_diff", "DIST_50", "DIST_98", "n1", "n2"])
                            # (3) Re-extract the conjunctive labels
                            for col in ["shape_semantic_1", "shape_semantic_2", "shape_semantic_same", "task_kind_1", "task_kind_2", "task_kind_same", "shape_semantic_12", "task_kind_12", "same-shape_semantic|task_kind"]:
                                if col in DFDISTS:
                                    del DFDISTS[col]
                            DFDISTS = dfdist_extract_label_vars_specific(DFDISTS, [var_effect, var_other])            

                        # Postprocess
                        DFDISTS["labels_pair_unique"] = [tuple(sorted((row["labels_1"], row["labels_2"]))) for _, row in DFDISTS.iterrows()]
                        DFDISTS["shapes_pair_unique"] = [tuple(sorted((row["labels_1"][0], row["labels_2"][0]))) for _, row in DFDISTS.iterrows()]
                        DFDISTS = _euclidianshuff_stats_linear_load_mult_dates_postprocess(DFDISTS)
                        
                        # Savedir
                        from pythonlib.tools.expttools import writeDictToTxt
                        writeDictToTxt(
                            {"list_date":list_date, 
                            "prune_version":prune_version,
                            "prune_version_unique_exists":DFDISTS["prune_version"].unique().tolist(),
                            "list_subspace":list_subspace,
                            }, 
                            f"{SAVEDIR_PLOTS}/params.txt")
                        _, _, _, _ = euclidianshuff_stats_linear_plot_wrapper(DFDISTS, SAVEDIR_PLOTS, var_other, 
                                                                              var_effect=var_effect, 
                                                                              subsample_plot_each_iter_stats = subsample_plot_each_iter_stats)



Loading:  /lemur2/lucas/analyses/recordings/main/euclidian_char_sp/EUCL_QUICK_SHUFFLE_revision-VAR_SHAPE=shape_semantic/Diego-231205-combine=True-wl=stroke-00_stroke-regrhack=False/list_dfdist.pkl
concatting...
appending...
agging over replicates...
agging over label pairs...
Loading:  /lemur2/lucas/analyses/recordings/main/euclidian_char_sp/EUCL_QUICK_SHUFFLE_revision-VAR_SHAPE=shape_semantic/Diego-231122-combine=True-wl=stroke-00_stroke-regrhack=False/list_dfdist.pkl
concatting...
appending...
agging over replicates...
agging over label pairs...
Loading:  /lemur2/lucas/analyses/recordings/main/euclidian_char_sp/EUCL_QUICK_SHUFFLE_revision-VAR_SHAPE=shape_semantic/Diego-231128-combine=True-wl=stroke-00_stroke-regrhack=False/list_dfdist.pkl
concatting...
appending...
agging over replicates...
agging over label pairs...
Loading:  /lemur2/lucas/analyses/recordings/main/euclidian_char_sp/EUCL_QUICK_SHUFFLE_revision-VAR_SHAPE=shape_semantic/Diego-231129-combine=True-wl=stroke-00_stroke-reg

/home/lucas/miniconda3/envs/drag2_matlab/lib/python3.8/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/lucas/miniconda3/envs/drag2_matlab/lib/python3.8/site-packages/numpy/core/_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Saved to:  /lemur2/lucas/analyses/recordings/main/euclidian_char_sp/EUCL_QUICK_SHUFFLE_revision-VAR_SHAPE=shape_semantic/MULT/stats_linear_model/char_sp_00_stroke_revision_noregr_seman/Diego-231205|231122|231128|231129|231201|231120|231206|231218|231220-var_other=task_kind-prune=sp_char_0-ss=['task_shape']-prunesh=False/scatter_each_label1/scatter_each_label1-grp=('00_stroke|(-0.3, 0.0)', 'task_shape|(-0.8, 0.3)')-counts.txt


/home/lucas/miniconda3/envs/drag2_matlab/lib/python3.8/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/lucas/miniconda3/envs/drag2_matlab/lib/python3.8/site-packages/numpy/core/_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Saved to:  /lemur2/lucas/analyses/recordings/main/euclidian_char_sp/EUCL_QUICK_SHUFFLE_revision-VAR_SHAPE=shape_semantic/MULT/stats_linear_model/char_sp_00_stroke_revision_noregr_seman/Diego-231205|231122|231128|231129|231201|231120|231206|231218|231220-var_other=task_kind-prune=sp_char_0-ss=['task_shape']-prunesh=False/scatter_each_label1/scatter_each_label1-grp=('00_stroke|(-0.35, -0.05)', 'task_shape|(-0.8, 0.3)')-counts.txt


...

Making copy and replacing None with 'none', for:  var_datapt
* This adds compute time!!!!
Saved to:  /lemur2/lucas/analyses/recordings/main/euclidian_char_sp/EUCL_QUICK_SHUFFLE_revision-VAR_SHAPE=shape_semantic/MULT/stats_linear_model/char_sp_00_stroke_revision_regr_seman/Pancho-220614|220616|220618|220621|220622|220624|220626|220627|220628|220630-var_other=task_kind-prune=sp_pig_1plus-ss=['task_shape']-prunesh=False/SUBSAMPLE/scatter/samplesize-confirmed_this_used_in_stats.txt
Saved to:  /lemur2/lucas/analyses/recordings/main/euclidian_char_sp/EUCL_QUICK_SHUFFLE_revision-VAR_SHAPE=shape_semantic/MULT/stats_linear_model/char_sp_00_stroke_revision_regr_seman/Pancho-220614|220616|220618|220621|220622|220624|220626|220627|220628|220630-var_other=task_kind-prune=sp_pig_1plus-ss=['task_shape']-prunesh=False/SUBSAMPLE/scatter/samplesize-confirmed_this_used_in_stats-split_by_date.txt
Saved to:  /lemur2/lucas/analyses/recordings/main/euclidian_char_sp/EUCL_QUICK_SHUFFLE_revision-VAR_SHAPE=shap

/home/lucas/miniconda3/envs/drag2_matlab/lib/python3.8/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


Saved to:  /lemur2/lucas/analyses/recordings/main/euclidian_char_sp/EUCL_QUICK_SHUFFLE_revision-VAR_SHAPE=shape_semantic/MULT/stats_linear_model/char_sp_00_stroke_revision_regr_seman/Pancho-220614|220616|220618|220621|220622|220624|220626|220627|220628|220630-var_other=task_kind-prune=sp_pig_1plus-ss=['task_shape']-prunesh=False/var_datapt=None-aggoverdates=False/linear_model_region_vs_region/samplesize-confirmed_this_used_in_stats.txt
Saved to:  /lemur2/lucas/analyses/recordings/main/euclidian_char_sp/EUCL_QUICK_SHUFFLE_revision-VAR_SHAPE=shape_semantic/MULT/stats_linear_model/char_sp_00_stroke_revision_regr_seman/Pancho-220614|220616|220618|220621|220622|220624|220626|220627|220628|220630-var_other=task_kind-prune=sp_pig_1plus-ss=['task_shape']-prunesh=False/var_datapt=None-aggoverdates=False/linear_model_region_vs_region/samplesize-confirmed_this_used_in_stats-split_by_date.txt
Saved to:  /lemur2/lucas/analyses/recordings/main/euclidian_char_sp/EUCL_QUICK_SHUFFLE_revision-VAR_SHAPE=

/home/lucas/miniconda3/envs/drag2_matlab/lib/python3.8/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


Saved to:  /lemur2/lucas/analyses/recordings/main/euclidian_char_sp/EUCL_QUICK_SHUFFLE_revision-VAR_SHAPE=shape_semantic/MULT/stats_linear_model/char_sp_00_stroke_revision_regr_seman/Pancho-220614|220616|220618|220621|220622|220624|220626|220627|220628|220630-var_other=task_kind-prune=sp_pig_1plus-ss=['task_shape']-prunesh=False/var_datapt=shapes_pair_unique-aggoverdates=False/linear_model_region_vs_region/samplesize-confirmed_this_used_in_stats.txt
Saved to:  /lemur2/lucas/analyses/recordings/main/euclidian_char_sp/EUCL_QUICK_SHUFFLE_revision-VAR_SHAPE=shape_semantic/MULT/stats_linear_model/char_sp_00_stroke_revision_regr_seman/Pancho-220614|220616|220618|220621|220622|220624|220626|220627|220628|220630-var_other=task_kind-prune=sp_pig_1plus-ss=['task_shape']-prunesh=False/var_datapt=shapes_pair_unique-aggoverdates=False/linear_model_region_vs_region/samplesize-confirmed_this_used_in_stats-split_by_date.txt
Saved to:  /lemur2/lucas/analyses/recordings/main/euclidian_char_sp/EUCL_QUICK

In [ ]:
_, _, _, _ = euclidianshuff_stats_linear_plot_wrapper(DFDISTS, SAVEDIR_PLOTS, var_other, var_effect=var_effect)


In [ ]:
from neuralmonkey.scripts.analy_shape_invariance_all_plots_SP import _euclidianshuff_stats_linear_2br_scatter_wrapper_SUBSAMPLE
SAVEDIR_PLOTS = "/tmp/ASDASD"
os.makedirs(SAVEDIR_PLOTS, exist_ok=True)
var_datapt = None
_euclidianshuff_stats_linear_2br_scatter_wrapper_SUBSAMPLE(DFDISTS, var_effect, var_same_same, var_datapt, SAVEDIR_PLOTS, plot_each_iter_stats=True)

##### Code here (devo)

In [ ]:
import os
SAVEDIR_ANALYSIS = "/tmp/CHAR_SP"
os.makedirs(SAVEDIR_ANALYSIS, exist_ok=True)


In [ ]:
# assert False, "save split, or else file gets to mult GB -- run seprately for each bregion."
var_effect="shape_semantic_grp"
var_conj = "task_kind"
vars_group = [var_effect, var_conj]
N_MIN_TRIALS_PER_SHAPE = 4
TWIND_ANALY = (-0.4, 0.5)
NPCS_KEEP = 6
DO_RSA_HEATMAPS = False

SUBSPACE_PROJ_FIT_TWIND = {
    "00_stroke":[(-0.5, -0.05), (0.05, 0.5), (-0.5, 0.5), (-0.4, 0.3)],
}

# LIST_SUBSPACE_PROJECTION = [None, "pca_proj", "task_shape_si", "shape_prims_single"]
LIST_SUBSPACE_PROJECTION = ["shape_prims_single"]
LIST_PRUNE_VERSION = ["sp_char"] # GOOD

N_SPLITS = 2

twind_analy = TWIND_ANALY
tbin_dur = 0.2
tbin_slide = 0.02

map_event_to_listtwind_scal = {
    # "00_stroke":[(-0.5, -0.05), (0.05, 0.5), (-0.3, 0.1)],
    # "00_stroke":[(-0.5, -0.05), (-0.3, 0.1), (-0.3, 0.2), (-0.4, 0.3), (-0.2, 0.3), (0.05, 0.5)],
    "00_stroke":[(0.05, 0.5)],
    }

list_dfdist =[]
for _, row in DFallpa.iterrows():
    bregion = row["bregion"]
    which_level = row["which_level"]
    event = row["event"]
    PA = row["pa"]

    for prune_version in LIST_PRUNE_VERSION:

        for subspace_projection in LIST_SUBSPACE_PROJECTION:
            # plot only cleaned up data.
            list_unstable_badstrokes = [(True, True, True)]
                
            # for remove_drift in [False]:
            for remove_drift, remove_singleprims_unstable, remove_trials_with_bad_strokes in list_unstable_badstrokes:

                ############################
                if subspace_projection in [None, "pca"]:
                    list_fit_twind = [twind_analy]
                else:
                    list_fit_twind = SUBSPACE_PROJ_FIT_TWIND[event]
                
                for subspace_projection_fitting_twind in list_fit_twind:
                    
                    # Final save dir
                    SAVEDIR = f"{SAVEDIR_ANALYSIS}/{which_level}-{bregion}-{event}--prune={prune_version}-ss={subspace_projection}-nodrift={remove_drift}-SpUnstable={remove_singleprims_unstable}-RmBadStrks={remove_trials_with_bad_strokes}-fit_twind={subspace_projection_fitting_twind}"
                    os.makedirs(SAVEDIR, exist_ok=True)
                    print("SAVING AT ... ", SAVEDIR)

                    if DO_RSA_HEATMAPS:
                        # Plot pairwise distances (rsa heatmaps).
                        # This is done separatee to below becuase it doesnt use the train-test splits.
                        # It shold but I would have to code way to merge multple Cl, which is doable.
                        from neuralmonkey.analyses.euclidian_distance import timevarying_compute_fast_to_scalar

                        # PAthis = preprocess_pa(PA, animal, date, var_other, None, remove_drift, 
                        #                        subspace_projection, subspace_projection_fitting_twind, 
                        #                        twind_analy, tbin_dur, tbin_slide, raw_subtract_mean_each_timepoint=False,
                        #                        skip_dim_reduction=False)
                        
                        PAthis = preprocess_pa(animal, date, PA, None, prune_version, 
                                            n_min_trials_per_shape=N_MIN_TRIALS_PER_SHAPE, shape_var=var_effect, plot_drawings=False,
                                            remove_chans_fr_drift=remove_drift, subspace_projection=subspace_projection, 
                                                twind_analy=twind_analy, tbin_dur=tbin_dur, tbin_slide=tbin_slide, NPCS_KEEP=NPCS_KEEP,
                                                raw_subtract_mean_each_timepoint=False, remove_singleprims_unstable=remove_singleprims_unstable,
                                                remove_trials_with_bad_strokes=remove_trials_with_bad_strokes, 
                                                subspace_projection_fitting_twind=subspace_projection_fitting_twind)
                        
                        list_twind_scalar = map_event_to_listtwind_scal[event]
                        for twind_scal in list_twind_scalar:
                            savedir = f"{SAVEDIR}/rsa_heatmap/twindscal={twind_scal}"
                            os.makedirs(savedir, exist_ok=True)

                            # Prune to scalar window
                            pa = PAthis.slice_by_dim_values_wrapper("times", twind_scal)

                            # Make rsa heatmaps.
                            timevarying_compute_fast_to_scalar(pa, vars_group, rsa_heatmap_savedir=savedir)

                    # Preprocess
                    savedir = f"{SAVEDIR}/preprocess"
                    os.makedirs(savedir, exist_ok=True)

                    skip_dim_reduction = True # will do so below... THis just do other preprocessing, and widowing
                    PAthis = preprocess_pa(animal, date, PA, savedir, prune_version, 
                                        n_min_trials_per_shape=N_MIN_TRIALS_PER_SHAPE, shape_var=var_effect, plot_drawings=False,
                                        remove_chans_fr_drift=remove_drift, subspace_projection=subspace_projection, 
                                            twind_analy=twind_analy, tbin_dur=tbin_dur, tbin_slide=tbin_slide, NPCS_KEEP=NPCS_KEEP,
                                            raw_subtract_mean_each_timepoint=False, remove_singleprims_unstable=remove_singleprims_unstable,
                                            remove_trials_with_bad_strokes=remove_trials_with_bad_strokes, 
                                            subspace_projection_fitting_twind=subspace_projection_fitting_twind,
                                            skip_dim_reduction=skip_dim_reduction)


                    ########### DO TRAIN-TEST SPLITS
                    folds_dflab = PAthis.split_balanced_stratified_kfold_subsample_level_of_var(vars_group, None, None, 
                                                                                                n_splits=N_SPLITS, 
                                                                                                do_balancing_of_train_inds=False)

                    for _i_dimredu, (train_inds, test_inds) in enumerate(folds_dflab):
                        # train_inds, more inds than than test_inds
                        train_inds = [int(i) for i in train_inds]
                        test_inds = [int(i) for i in test_inds]

                        ############# DO DIM REDUCTION
                        from neuralmonkey.scripts.analy_shape_invariance_all_plots_SP import _preprocess_pa_dim_reduction
                        PAthisRedu = _preprocess_pa_dim_reduction(PAthis, subspace_projection, subspace_projection_fitting_twind,
                                twind_analy, tbin_dur, tbin_slide, savedir=None, raw_subtract_mean_each_timepoint=False,
                                inds_pa_fit=test_inds, inds_pa_final=train_inds)

                        if PAthisRedu is None:
                            print("SKIPPING, since PAthisRedu is None: ", SAVEDIR)
                            assert False
                        else:
                            # Take different windows (for computing scalar score)
                            # Go thru diff averaging windows (to get scalar)
                            list_twind_scalar = map_event_to_listtwind_scal[event]
                            for twind_scal in list_twind_scalar:
                                
                                pa = PAthisRedu.slice_by_dim_values_wrapper("times", twind_scal)

                                # ###################################### Running euclidian
                                # from neuralmonkey.analyses.euclidian_distance import timevarying_compute_fast_to_scalar
                                
                                # # (1) Data
                                # dfdist, _ = timevarying_compute_fast_to_scalar(pa, label_vars=vars_group)
                                
                                # dfdist["bregion"] = bregion
                                # dfdist["prune_version"] = prune_version
                                # dfdist["which_level"] = which_level
                                # dfdist["event"] = event
                                # dfdist["subspace_projection"] = subspace_projection
                                # dfdist["subspace_projection_fitting_twind"] = [subspace_projection_fitting_twind for _ in range(len(dfdist))]
                                # dfdist["dim_redu_fold"] = _i_dimredu
                                # dfdist["twind_scal"] = [twind_scal for _ in range(len(dfdist))]
                                # list_dfdist.append(dfdist)


In [ ]:
dflab = PAthis.Xlabels["trials"]

from pythonlib.tools.pandastools import grouping_plot_n_samples_conjunction_heatmap
grouping_plot_n_samples_conjunction_heatmap(dflab, "shape_semantic_grp", "task_kind", ["stroke_index"]);

In [ ]:
savedir = "/tmp"

In [ ]:
subspace_projection

In [ ]:
subspace_projection_fitting_twindsubspace_projection_fitting_twind

In [ ]:
PAthisRedu = _preprocess_pa_dim_reduction(PAthis, subspace_projection, subspace_projection_fitting_twind,
        twind_analy, tbin_dur, tbin_slide, savedir=None, raw_subtract_mean_each_timepoint=False,
        inds_pa_fit=test_inds, inds_pa_final=train_inds)
print(PAthisRedu)

# [OLDER] Euclidian timecourse

In [ ]:
import os
SAVEDIR_ANALYSIS = f"/tmp/EUCL_TIMECOURSE"
os.makedirs(SAVEDIR_ANALYSIS, exist_ok=True)
from neuralmonkey.scripts.analy_euclidian_chars_sp import euclidian_time_resolved_wrapper
euclidian_time_resolved_wrapper(animal, date, DFallpa, SAVEDIR_ANALYSIS)

# Cross-temporal scoring

Consider each timebin for char vs. each timebin for SP.

For each, get scores of diff|same, etc.


In [ ]:

# - Reshape PA so that time is on the trial axis.
from neuralmonkey.classes.population import concatenate_popanals

list_pa = []
list_labels = []
for i in range(len(PAredu.Times)):
    pathis = PAredu.slice_by_dim_indices_wrapper("times", [i])

    # collect infoextract_snippets_trials
    list_pa.append(pathis)
    list_labels.append(i)

PAreduScal = concatenate_popanals(list_pa, dim="trials", 
                                map_idxpa_to_value=list_labels, 
                                map_idxpa_to_value_colname="time_bin",
                                assert_otherdims_have_same_values=False,
                                times_realign_so_first_index_is_this_time=0)
PAreduScal.Times

In [ ]:
Cldist = PAreduScal.dataextract_as_distance_matrix_clusters_flex(["task_kind", "shape_semantic", "time_bin"], 
                                                             return_as_single_mean_over_time=True)



In [ ]:
dfres = Cldist.rsa_distmat_score_all_pairs_of_label_groups(False)


In [ ]:
dfres["task_kind_same"] = dfres["task_kind_1"] == dfres["task_kind_2"]
dfres["shape_semantic_same"] = dfres["shape_semantic_1"] == dfres["shape_semantic_2"]
dfres = append_col_with_grp_index(dfres, ["shape_semantic_same", "task_kind_same"], "shape_task_same")
dfres = append_col_with_grp_index(dfres, ["task_kind_1", "task_kind_2"], "task_kind_pair")
dfres = append_col_with_grp_index(dfres, ["shape_semantic_1", "shape_semantic_2"], "shape_semantic_pair")
dfres = append_col_with_grp_index(dfres, ["time_bin_1", "time_bin_2"], "time_bin_pair")

dfres = append_col_with_grp_index(dfres, ["shape_semantic_same", "task_kind_1", "task_kind_2"], "ss_task_kind_pair")


In [ ]:
dfres

In [ ]:
plot_subplots_heatmap(dfres, "time_bin_1", "time_bin_2", "dist_yue_diff", "ss_task_kind_pair", share_zlim=True)
plot_subplots_heatmap(dfres, "time_bin_1", "time_bin_2", "dist_mean", "ss_task_kind_pair", share_zlim=True)


In [ ]:
# TODO: Take difference across subplots, plotting a new subplot.

# Check motor similarity of strokes from SP vs. CHAR

In [ ]:
from neuralmonkey.scripts.analy_euclidian_chars_sp import preprocess_pa
from neuralmonkey.classes.population_mult import extract_single_pa

savedir = "/tmp/PREPROCESS"
os.makedirs(savedir, exist_ok=True)
PA = preprocess_pa(PA, savedir)

# Collect and plot across days

In [ ]:
SAVEDIR_SAVE = f"/lemur2/lucas/analyses/recordings/main/euclidian_char_sp/MULT"

list_animal_date = [
    # ("Pancho", 230112),
    # ("Pancho", 230117),
    # ("Pancho", 230118),
    ("Pancho", 230119),
    ("Pancho", 230120),
    ("Pancho", 230122),
    ("Pancho", 230125),
    ("Pancho", 230126),
    ("Pancho", 230127),
    ]

combine = True
ANIMAL = "Pancho"

LIST_NPCS_KEEP = [4,6,2]
for twind_analy in [(0.05, 0.25), (-0.05, 0.35), (0.1, 0.2)]:
    for subspace_projection in ["shape_prims_single", "pca"]:
        for prune_version in ["sp_char_0", "sp_char"]:
            for NPCS_KEEP in LIST_NPCS_KEEP:
                for raw_subtract_mean_each_timepoint in [False, True]:

                    # subspace_projection = "shape_prims_single"
                    # prune_version = "sp_char_0"
                    # NPCS_KEEP = 4
                    # raw_subtract_mean_each_timepoint = False
                    # twind_analy = (0.05, 0.25)

                    try:
                        ### Load all data for this params configuration
                        list_dfres = []
                        for animal, date in list_animal_date:
                            if animal==ANIMAL:
                                # animal = "Pancho"
                                # date = 230126

                                SAVEDIR_ANALYSIS = f"/lemur2/lucas/analyses/recordings/main/euclidian_char_sp/{animal}-{date}-combine={combine}"
                                path = f"{SAVEDIR_ANALYSIS}/subspc={subspace_projection}-prunedat={prune_version}-npcs={NPCS_KEEP}-subtr={raw_subtract_mean_each_timepoint}-twind={twind_analy}/summary/DFRES.pkl"
                                dfres = pd.read_pickle(path)

                                dfres["animal"] = animal
                                dfres["date"] = date

                                list_dfres.append(dfres)
                    except FileNotFoundError as err:
                        print("Skipping, did not find all data...", path)
                        continue

                    ######### PLOTS
                    savedir = f"{SAVEDIR_SAVE}/{ANIMAL}/combine={combine}-subspc={subspace_projection}-prunedat={prune_version}-npcs={NPCS_KEEP}-subtr={raw_subtract_mean_each_timepoint}-twind={twind_analy}"
                    os.makedirs(savedir, exist_ok=True)

                    DFRES = pd.concat(list_dfres).reset_index(drop=True)

                    from pythonlib.tools.pandastools import append_col_with_grp_index
                    DFRES = append_col_with_grp_index(DFRES, ["animal", "date"], "ani_dat")

                    from pythonlib.tools.pandastools import plot_45scatter_means_flexible_grouping
                    _, fig = plot_45scatter_means_flexible_grouping(DFRES, "shape_task_same", "1|0", "0|1", "bregion", 
                                                                    "dist_yue_diff", "ani_dat", True, SIZE=3, shareaxes=True);
                    savefig(fig, f"{savedir}/scatter-1.pdf")

                    _, fig = plot_45scatter_means_flexible_grouping(DFRES, "shape_task_same", "1|0", "0|1", None, 
                                                                    "dist_yue_diff", "bregion", True, SIZE=4, shareaxes=True);
                    savefig(fig, f"{savedir}/scatter-2.pdf")

                    _, fig = plot_45scatter_means_flexible_grouping(DFRES, "shape_task_same", "1|0", "0|1", "bregion", 
                                                                    "dist_yue_diff", "ani_dat", False, SIZE=6, shareaxes=True,
                                                                    color_by_var_datapt=True, alpha=0.5,
                                                                    force_all_on_same_axis=True);
                    savefig(fig, f"{savedir}/scatter-3.pdf")

                    fig = sns.catplot(data=DFRES, x = "bregion", hue="shape_task_same", y="dist_yue_diff", kind="bar", aspect=2)
                    savefig(fig, f"{savedir}/catplot-1.pdf")

                    fig = sns.catplot(data=DFRES, x = "bregion", hue="ani_dat", y="dist_yue_diff", kind="point", aspect=2, col="shape_task_same", col_wrap=2)
                    savefig(fig, f"{savedir}/catplot-2.pdf")


                    plt.close("all")

In [ ]:
# [MULT DAYS]